In [3]:
import pandas as pd
import numpy as np 
import datetime
from datetime import date
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd
from functools import reduce

import warnings
warnings.filterwarnings("ignore")

## Data constants 

In [4]:
bus_dates = ['2019-07-31',
             '2019-09-30',
             '2019-10-31',
             '2019-12-31'
#              '2019-03-31'
            ]
# df_mom_0_30['DT_BUSINESSDATE'].head()

In [5]:
#### =6 means all the accounts with at least 6 MOB as of snapshot 
min_perf_AO_snapshot = 6

### import MoM data

In [6]:
df_mom = pd.read_csv('D:/Varthana/MOM_DATA_28-Sep-20.csv')

##### Convert date variable to date format

In [7]:
df_mom['DT_BUSINESSDATE'] = pd.to_datetime(df_mom['DT_BUSINESSDATE'])
df_mom['DT_INSTALLMENTDUE'] = pd.to_datetime(df_mom['DT_INSTALLMENTDUE'])
df_mom['DT_LAST_PAYMENT'] = pd.to_datetime(df_mom['DT_LAST_PAYMENT'])
df_mom['DT_CREATED'] = pd.to_datetime(df_mom['DT_CREATED'])
df_mom['DT_LAST_UPDATE'] = pd.to_datetime(df_mom['DT_LAST_UPDATE'])


### Adding disb date to MoM data

In [8]:
LMS_det_all = pd.read_csv('D:/Varthana/LMS_DETAILS_11-Sep-20.csv')

In [9]:
LMS_det_all_1 = pd.DataFrame(LMS_det_all[['LOAN_ID',
                                          'FIRST_DISB_DATE',
                                          'TENOR',
                                          'START_DATE',
                                          'END_DATE',
                                         'LOAN_AMOUNT']])

In [10]:
LMS_det_all_1['FIRST_DISB_DATE'] = pd.to_datetime(LMS_det_all_1['FIRST_DISB_DATE'])
LMS_det_all_1['START_DATE'] = pd.to_datetime(LMS_det_all_1['START_DATE'])
LMS_det_all_1['END_DATE'] = pd.to_datetime(LMS_det_all_1['END_DATE'])

In [11]:
df_mom_2  = pd.merge(df_mom, LMS_det_all_1,how="left",left_on="SZ_LOAN_ACCOUNT_NO", right_on="LOAN_ID")

In [12]:
## disb date not available
list(df_mom_2[df_mom_2['FIRST_DISB_DATE'].isnull()]['SZ_LOAN_ACCOUNT_NO'].unique())

['PUR7108U',
 'S18BLR-BLR-002142',
 'S19NAS-NAS-007192',
 'S20NAG-GON-013978',
 'U18HYD-HYD-000835']

## creating delinq flag vars

In [13]:
df_mom_2["flag_30p"] = np.where(df_mom_2["I_DPD"] >=30,1,0)
df_mom_2["flag_60p"] = np.where(df_mom_2["I_DPD"] >=60,1,0)
df_mom_2["flag_90p"] = np.where(df_mom_2["I_DPD"] >=90,1,0)

### creating Days late payment var & delq vars

In [14]:
df_mom_2['days_late_paid'] = (df_mom_2['DT_LAST_PAYMENT'] - df_mom_2['DT_INSTALLMENTDUE']).dt.days

In [15]:
df_mom_2['late_paymt_flag'] = np.where(df_mom_2['DT_LAST_PAYMENT']> df_mom_2['DT_INSTALLMENTDUE'],1,0)

##### Lag variables creation

In [16]:
df_mom_2 = df_mom_2.sort_values(["SZ_LOAN_ACCOUNT_NO","DT_BUSINESSDATE"],ascending=[True,True] )

In [17]:
df_mom_2['late_paymt_flag_lag_1'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['late_paymt_flag'].shift(1)
df_mom_2['late_paymt_flag_lag_2'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['late_paymt_flag'].shift(2)
df_mom_2['late_paymt_flag_lag_3'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['late_paymt_flag'].shift(3)
df_mom_2['late_paymt_flag_lag_4'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['late_paymt_flag'].shift(4)
df_mom_2['late_paymt_flag_lag_5'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['late_paymt_flag'].shift(5)
df_mom_2['late_paymt_flag_lag_6'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['late_paymt_flag'].shift(6)
df_mom_2['late_paymt_flag_lag_7'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['late_paymt_flag'].shift(7)
df_mom_2['late_paymt_flag_lag_8'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['late_paymt_flag'].shift(8)
df_mom_2['late_paymt_flag_lag_9'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['late_paymt_flag'].shift(9)
df_mom_2['late_paymt_flag_lag_10'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['late_paymt_flag'].shift(10)
df_mom_2['late_paymt_flag_lag_11'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['late_paymt_flag'].shift(11)
df_mom_2['late_paymt_flag_lag_12'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['late_paymt_flag'].shift(12)

In [18]:
df_mom_2['days_late_paid_lag_1'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['days_late_paid'].shift(1)
df_mom_2['days_late_paid_lag_2'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['days_late_paid'].shift(2)
df_mom_2['days_late_paid_lag_3'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['days_late_paid'].shift(3)
df_mom_2['days_late_paid_lag_4'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['days_late_paid'].shift(4)
df_mom_2['days_late_paid_lag_5'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['days_late_paid'].shift(5)
df_mom_2['days_late_paid_lag_6'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['days_late_paid'].shift(6)
df_mom_2['days_late_paid_lag_7'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['days_late_paid'].shift(7)
df_mom_2['days_late_paid_lag_8'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['days_late_paid'].shift(8)
df_mom_2['days_late_paid_lag_9'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['days_late_paid'].shift(9)
df_mom_2['days_late_paid_lag_10'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['days_late_paid'].shift(10)
df_mom_2['days_late_paid_lag_11'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['days_late_paid'].shift(11)
df_mom_2['days_late_paid_lag_12'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['days_late_paid'].shift(12)

In [19]:
df_mom_2['xdays_flag_lag1'] = np.where((df_mom_2['days_late_paid_lag_1'] >=1) & (df_mom_2['days_late_paid_lag_1']<=29),1,0)
df_mom_2['xdays_flag_lag2'] = np.where((df_mom_2['days_late_paid_lag_2'] >=1) & (df_mom_2['days_late_paid_lag_2']<=29),1,0)
df_mom_2['xdays_flag_lag3'] = np.where((df_mom_2['days_late_paid_lag_3'] >=1) & (df_mom_2['days_late_paid_lag_3']<=29),1,0)
df_mom_2['xdays_flag_lag4'] = np.where((df_mom_2['days_late_paid_lag_4'] >=1) & (df_mom_2['days_late_paid_lag_4']<=29),1,0)
df_mom_2['xdays_flag_lag5'] = np.where((df_mom_2['days_late_paid_lag_5'] >=1) & (df_mom_2['days_late_paid_lag_5']<=29),1,0)
df_mom_2['xdays_flag_lag6'] = np.where((df_mom_2['days_late_paid_lag_6'] >=1) & (df_mom_2['days_late_paid_lag_6']<=29),1,0)
df_mom_2['xdays_flag_lag7'] = np.where((df_mom_2['days_late_paid_lag_7'] >=1) & (df_mom_2['days_late_paid_lag_7']<=29),1,0)
df_mom_2['xdays_flag_lag8'] = np.where((df_mom_2['days_late_paid_lag_8'] >=1) & (df_mom_2['days_late_paid_lag_8']<=29),1,0)
df_mom_2['xdays_flag_lag9'] = np.where((df_mom_2['days_late_paid_lag_9'] >=1) & (df_mom_2['days_late_paid_lag_9']<=29),1,0)
df_mom_2['xdays_flag_lag10'] = np.where((df_mom_2['days_late_paid_lag_10'] >=1) & (df_mom_2['days_late_paid_lag_10']<=29),1,0)
df_mom_2['xdays_flag_lag11'] = np.where((df_mom_2['days_late_paid_lag_11'] >=1) & (df_mom_2['days_late_paid_lag_11']<=29),1,0)
df_mom_2['xdays_flag_lag12'] = np.where((df_mom_2['days_late_paid_lag_12'] >=1) & (df_mom_2['days_late_paid_lag_12']<=29),1,0)


In [20]:
df_mom_2['dpd_lag_1'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['I_DPD'].shift(1)
df_mom_2['dpd_lag_2'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['I_DPD'].shift(2)
df_mom_2['dpd_lag_3'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['I_DPD'].shift(3)
df_mom_2['dpd_lag_4'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['I_DPD'].shift(4)
df_mom_2['dpd_lag_5'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['I_DPD'].shift(5)
df_mom_2['dpd_lag_6'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['I_DPD'].shift(6)
df_mom_2['dpd_lag_7'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['I_DPD'].shift(7)
df_mom_2['dpd_lag_8'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['I_DPD'].shift(8)
df_mom_2['dpd_lag_9'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['I_DPD'].shift(9)
df_mom_2['dpd_lag_10'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['I_DPD'].shift(10)
df_mom_2['dpd_lag_11'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['I_DPD'].shift(11)
df_mom_2['dpd_lag_12'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['I_DPD'].shift(12)

In [21]:
df_mom_2['dpd_30pflag_lag1'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_30p'].shift(1)
df_mom_2['dpd_30pflag_lag2'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_30p'].shift(2)
df_mom_2['dpd_30pflag_lag3'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_30p'].shift(3)
df_mom_2['dpd_30pflag_lag4'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_30p'].shift(4)
df_mom_2['dpd_30pflag_lag5'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_30p'].shift(5)
df_mom_2['dpd_30pflag_lag6'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_30p'].shift(6)
df_mom_2['dpd_30pflag_lag7'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_30p'].shift(7)
df_mom_2['dpd_30pflag_lag8'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_30p'].shift(8)
df_mom_2['dpd_30pflag_lag9'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_30p'].shift(9)
df_mom_2['dpd_30pflag_lag10'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_30p'].shift(10)
df_mom_2['dpd_30pflag_lag11'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_30p'].shift(11)
df_mom_2['dpd_30pflag_lag12'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_30p'].shift(12)


In [22]:
df_mom_2['dpd_60pflag_lag1'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_60p'].shift(1)
df_mom_2['dpd_60pflag_lag2'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_60p'].shift(2)
df_mom_2['dpd_60pflag_lag3'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_60p'].shift(3)
df_mom_2['dpd_60pflag_lag4'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_60p'].shift(4)
df_mom_2['dpd_60pflag_lag5'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_60p'].shift(5)
df_mom_2['dpd_60pflag_lag6'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_60p'].shift(6)
df_mom_2['dpd_60pflag_lag7'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_60p'].shift(7)
df_mom_2['dpd_60pflag_lag8'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_60p'].shift(8)
df_mom_2['dpd_60pflag_lag9'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_60p'].shift(9)
df_mom_2['dpd_60pflag_lag10'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_60p'].shift(10)
df_mom_2['dpd_60pflag_lag11'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_60p'].shift(11)
df_mom_2['dpd_60pflag_lag12'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_60p'].shift(12)

In [23]:
df_mom_2['dpd_90pflag_lag1'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_90p'].shift(1)
df_mom_2['dpd_90pflag_lag2'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_90p'].shift(2)
df_mom_2['dpd_90pflag_lag3'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_90p'].shift(3)
df_mom_2['dpd_90pflag_lag4'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_90p'].shift(4)
df_mom_2['dpd_90pflag_lag5'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_90p'].shift(5)
df_mom_2['dpd_90pflag_lag6'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_90p'].shift(6)
df_mom_2['dpd_90pflag_lag7'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_90p'].shift(7)
df_mom_2['dpd_90pflag_lag8'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_90p'].shift(8)
df_mom_2['dpd_90pflag_lag9'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_90p'].shift(9)
df_mom_2['dpd_90pflag_lag10'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_90p'].shift(10)
df_mom_2['dpd_90pflag_lag11'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_90p'].shift(11)
df_mom_2['dpd_90pflag_lag12'] = df_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO'])['flag_90p'].shift(12)

# Times xdays, 30+,60+,90+ variables

##### Times 30+,60+,90+ in last 12mths

In [24]:
df_mom_2['Times_30p_12m'] = df_mom_2.loc[:,["dpd_30pflag_lag1",
                                            "dpd_30pflag_lag2",
                                            "dpd_30pflag_lag3",
                                            "dpd_30pflag_lag4",
                                            "dpd_30pflag_lag5",
                                            "dpd_30pflag_lag6",
                                            "dpd_30pflag_lag7",
                                            "dpd_30pflag_lag8",
                                            "dpd_30pflag_lag9",
                                            "dpd_30pflag_lag10",
                                            "dpd_30pflag_lag11",
                                            "dpd_30pflag_lag12"
                                            
                                           ]].sum(axis=1)


In [25]:
df_mom_2['Times_60p_12m'] = df_mom_2.loc[:,["dpd_60pflag_lag1",
                                            "dpd_60pflag_lag2",
                                            "dpd_60pflag_lag3",
                                            "dpd_60pflag_lag4",
                                            "dpd_60pflag_lag5",
                                            "dpd_60pflag_lag6",
                                            "dpd_60pflag_lag7",
                                            "dpd_60pflag_lag8",
                                            "dpd_60pflag_lag9",
                                            "dpd_60pflag_lag10",
                                            "dpd_60pflag_lag11",
                                            "dpd_60pflag_lag12"
                                            
                                           ]].sum(axis=1)

In [26]:
df_mom_2['Times_90p_12m'] = df_mom_2.loc[:,["dpd_90pflag_lag1",
                                            "dpd_90pflag_lag2",
                                            "dpd_90pflag_lag3",
                                            "dpd_90pflag_lag4",
                                            "dpd_90pflag_lag5",
                                            "dpd_90pflag_lag6",
                                            "dpd_90pflag_lag7",
                                            "dpd_90pflag_lag8",
                                            "dpd_90pflag_lag9",
                                            "dpd_90pflag_lag10",
                                            "dpd_90pflag_lag11",
                                            "dpd_90pflag_lag12"
                                            
                                           ]].sum(axis=1)

In [27]:
df_mom_2['Times_xdays_12m'] = df_mom_2.loc[:,['xdays_flag_lag1',
                                              'xdays_flag_lag2',
                                              'xdays_flag_lag3',
                                              'xdays_flag_lag4',
                                              'xdays_flag_lag5',
                                              'xdays_flag_lag6',
                                              'xdays_flag_lag7',
                                              'xdays_flag_lag8',
                                              'xdays_flag_lag9',
                                              'xdays_flag_lag10',
                                              'xdays_flag_lag11',
                                              'xdays_flag_lag12'
                                             ]].sum(axis=1)

##### Times 30+,60+,90+ in last 9mths

In [28]:
df_mom_2['Times_30p_9m'] = df_mom_2.loc[:,["dpd_30pflag_lag1",
                                            "dpd_30pflag_lag2",
                                            "dpd_30pflag_lag3",
                                            "dpd_30pflag_lag4",
                                            "dpd_30pflag_lag5",
                                            "dpd_30pflag_lag6",
                                            "dpd_30pflag_lag7",
                                            "dpd_30pflag_lag8",
                                            "dpd_30pflag_lag9"
                                            
                                           ]].sum(axis=1)

In [29]:
df_mom_2['Times_60p_9m'] = df_mom_2.loc[:,["dpd_60pflag_lag1",
                                            "dpd_60pflag_lag2",
                                            "dpd_60pflag_lag3",
                                            "dpd_60pflag_lag4",
                                            "dpd_60pflag_lag5",
                                            "dpd_60pflag_lag6",
                                            "dpd_60pflag_lag7",
                                            "dpd_60pflag_lag8",
                                            "dpd_60pflag_lag9"
                                            
                                           ]].sum(axis=1)

In [30]:
df_mom_2['Times_90p_9m'] = df_mom_2.loc[:,["dpd_90pflag_lag1",
                                            "dpd_90pflag_lag2",
                                            "dpd_90pflag_lag3",
                                            "dpd_90pflag_lag4",
                                            "dpd_90pflag_lag5",
                                            "dpd_90pflag_lag6",
                                            "dpd_90pflag_lag7",
                                            "dpd_90pflag_lag8",
                                            "dpd_90pflag_lag9"
                                           ]].sum(axis=1)

In [31]:
df_mom_2['Times_xdays_9m'] = df_mom_2.loc[:,['xdays_flag_lag1',
                                              'xdays_flag_lag2',
                                              'xdays_flag_lag3',
                                              'xdays_flag_lag4',
                                              'xdays_flag_lag5',
                                              'xdays_flag_lag6',
                                              'xdays_flag_lag7',
                                              'xdays_flag_lag8',
                                              'xdays_flag_lag9'
                                             ]].sum(axis=1)

##### Times 30+,60+,90+ in last 6mths

In [32]:
df_mom_2['Times_30p_6m'] = df_mom_2.loc[:,["dpd_30pflag_lag1",
                                            "dpd_30pflag_lag2",
                                            "dpd_30pflag_lag3",
                                            "dpd_30pflag_lag4",
                                            "dpd_30pflag_lag5",
                                            "dpd_30pflag_lag6"
                                                                                       
                                           ]].sum(axis=1)

In [33]:
df_mom_2['Times_60p_6m'] = df_mom_2.loc[:,["dpd_60pflag_lag1",
                                            "dpd_60pflag_lag2",
                                            "dpd_60pflag_lag3",
                                            "dpd_60pflag_lag4",
                                            "dpd_60pflag_lag5",
                                            "dpd_60pflag_lag6"
                                          
                                           ]].sum(axis=1)

In [34]:
df_mom_2['Times_90p_6m'] = df_mom_2.loc[:,["dpd_90pflag_lag1",
                                            "dpd_90pflag_lag2",
                                            "dpd_90pflag_lag3",
                                            "dpd_90pflag_lag4",
                                            "dpd_90pflag_lag5",
                                            "dpd_90pflag_lag6"
                                         
                                           ]].sum(axis=1)

In [35]:
df_mom_2['Times_xdays_6m'] = df_mom_2.loc[:,['xdays_flag_lag1',
                                              'xdays_flag_lag2',
                                              'xdays_flag_lag3',
                                              'xdays_flag_lag4',
                                              'xdays_flag_lag5',
                                              'xdays_flag_lag6'
                                             ]].sum(axis=1)

##### Times 30+,60+,90+ in last 3mths

In [36]:
df_mom_2['Times_30p_3m'] = df_mom_2.loc[:,["dpd_30pflag_lag1",
                                            "dpd_30pflag_lag2",
                                            "dpd_30pflag_lag3"                        
                                           ]].sum(axis=1)

In [37]:
df_mom_2['Times_60p_3m'] = df_mom_2.loc[:,["dpd_60pflag_lag1",
                                            "dpd_60pflag_lag2",
                                            "dpd_60pflag_lag3"
                                           ]].sum(axis=1)

In [38]:
df_mom_2['Times_90p_3m'] = df_mom_2.loc[:,["dpd_90pflag_lag1",
                                            "dpd_90pflag_lag2",
                                            "dpd_90pflag_lag3"
                                           ]].sum(axis=1)

In [39]:
df_mom_2['Times_xdays_3m'] = df_mom_2.loc[:,['xdays_flag_lag1',
                                              'xdays_flag_lag2',
                                              'xdays_flag_lag3'
                                             ]].sum(axis=1)

# Flag - 30p,60p,90p

##### Flag - 30p,60p,90p in 12 mths 

In [40]:
df_mom_2['Last_12mths_30p_flag'] = np.where(df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3',
                                                   'dpd_lag_4',
                                                   'dpd_lag_5',
                                                   'dpd_lag_6',
                                                   'dpd_lag_7',
                                                   'dpd_lag_8',
                                                   'dpd_lag_9',
                                                   'dpd_lag_10',
                                                    'dpd_lag_11',
                                                    'dpd_lag_12'
                                                 ]].max(axis=1) >=30,1,0)



In [41]:
df_mom_2['Last_12mths_60p_flag'] = np.where(df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3',
                                                   'dpd_lag_4',
                                                   'dpd_lag_5',
                                                   'dpd_lag_6',
                                                   'dpd_lag_7',
                                                   'dpd_lag_8',
                                                   'dpd_lag_9',
                                                   'dpd_lag_10',
                                                    'dpd_lag_11',
                                                    'dpd_lag_12'
                                                 ]].max(axis=1) >=60,1,0)

In [42]:
df_mom_2['Last_12mths_90p_flag'] = np.where(df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3',
                                                   'dpd_lag_4',
                                                   'dpd_lag_5',
                                                   'dpd_lag_6',
                                                   'dpd_lag_7',
                                                   'dpd_lag_8',
                                                   'dpd_lag_9',
                                                   'dpd_lag_10',
                                                    'dpd_lag_11',
                                                    'dpd_lag_12'
                                                 ]].max(axis=1) >=90,1,0)

In [43]:
df_mom_2['Last_12mths_xdays_flag'] = np.where(df_mom_2.loc[:,['xdays_flag_lag1',
                                              'xdays_flag_lag2',
                                              'xdays_flag_lag3',
                                              'xdays_flag_lag4',
                                              'xdays_flag_lag5',
                                              'xdays_flag_lag6',
                                              'xdays_flag_lag7',
                                              'xdays_flag_lag8',
                                              'xdays_flag_lag9',
                                              'xdays_flag_lag10',
                                              'xdays_flag_lag11',
                                              'xdays_flag_lag12'
                                             ]].max(axis=1)==1,1,0)

##### Flag - 30p,60p,90p in 9 mths 

In [44]:
df_mom_2['Last_9mths_30p_flag'] = np.where(df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3',
                                                   'dpd_lag_4',
                                                   'dpd_lag_5',
                                                   'dpd_lag_6',
                                                   'dpd_lag_7',
                                                   'dpd_lag_8',
                                                   'dpd_lag_9'
                                                 ]].max(axis=1) >=30,1,0)



In [45]:
df_mom_2['Last_9mths_60p_flag'] = np.where(df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3',
                                                   'dpd_lag_4',
                                                   'dpd_lag_5',
                                                   'dpd_lag_6',
                                                   'dpd_lag_7',
                                                   'dpd_lag_8',
                                                   'dpd_lag_9'
                                                 ]].max(axis=1) >=60,1,0)

In [46]:
df_mom_2['Last_9mths_90p_flag'] = np.where(df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3',
                                                   'dpd_lag_4',
                                                   'dpd_lag_5',
                                                   'dpd_lag_6',
                                                   'dpd_lag_7',
                                                   'dpd_lag_8',
                                                   'dpd_lag_9'
                                                 ]].max(axis=1) >=90,1,0)

In [47]:
df_mom_2['Last_9mths_xdays_flag'] = np.where(df_mom_2.loc[:,['xdays_flag_lag1',
                                              'xdays_flag_lag2',
                                              'xdays_flag_lag3',
                                              'xdays_flag_lag4',
                                              'xdays_flag_lag5',
                                              'xdays_flag_lag6',
                                              'xdays_flag_lag7',
                                              'xdays_flag_lag8',
                                              'xdays_flag_lag9'
                                             ]].max(axis=1)==1,1,0)

##### Flag - 30p,60p,90p in 6 mths 

In [48]:
df_mom_2['Last_6mths_30p_flag'] = np.where(df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3',
                                                   'dpd_lag_4',
                                                   'dpd_lag_5',
                                                   'dpd_lag_6'
                                                 ]].max(axis=1) >=30,1,0)

In [49]:
df_mom_2['Last_6mths_60p_flag'] = np.where(df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3',
                                                   'dpd_lag_4',
                                                   'dpd_lag_5',
                                                   'dpd_lag_6'
                                                 ]].max(axis=1) >=60,1,0)

In [50]:
df_mom_2['Last_6mths_90p_flag'] = np.where(df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3',
                                                   'dpd_lag_4',
                                                   'dpd_lag_5',
                                                   'dpd_lag_6'
                                                 ]].max(axis=1) >=90,1,0)

In [51]:
df_mom_2['Last_6mths_xdays_flag'] = np.where(df_mom_2.loc[:,['xdays_flag_lag1',
                                              'xdays_flag_lag2',
                                              'xdays_flag_lag3',
                                              'xdays_flag_lag4',
                                              'xdays_flag_lag5',
                                              'xdays_flag_lag6'
                                             ]].max(axis=1)==1,1,0)

##### Flag - 30p,60p,90p in 3 mths 

In [52]:
df_mom_2['Last_3mths_30p_flag'] = np.where(df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3'
                                                 ]].max(axis=1) >=30,1,0)

In [53]:
df_mom_2['Last_3mths_60p_flag'] = np.where(df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3'
                                                 ]].max(axis=1) >=60,1,0)

In [54]:
df_mom_2['Last_3mths_90p_flag'] = np.where(df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3'
                                                 ]].max(axis=1) >=90,1,0)

In [55]:
df_mom_2['Last_3mths_xdays_flag'] = np.where(df_mom_2.loc[:,['xdays_flag_lag1',
                                              'xdays_flag_lag2',
                                              'xdays_flag_lag3'
                                             ]].max(axis=1)==1,1,0)

# Max delinq last 12mths/9mths/6mths/3mths

In [56]:
df_mom_2['Max_dpd_Last_12m'] = df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3',
                                                   'dpd_lag_4',
                                                   'dpd_lag_5',
                                                   'dpd_lag_6',
                                                   'dpd_lag_7',
                                                   'dpd_lag_8',
                                                   'dpd_lag_9',
                                                   'dpd_lag_10',
                                                    'dpd_lag_11',
                                                    'dpd_lag_12'
                                                 ]].max(axis=1) 

In [57]:
df_mom_2['Max_dpd_Last_9m'] = df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3',
                                                   'dpd_lag_4',
                                                   'dpd_lag_5',
                                                   'dpd_lag_6',
                                                   'dpd_lag_7',
                                                   'dpd_lag_8',
                                                   'dpd_lag_9'
                                                 ]].max(axis=1) 

In [58]:
df_mom_2['Max_dpd_Last_6m'] = df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3',
                                                   'dpd_lag_4',
                                                   'dpd_lag_5',
                                                   'dpd_lag_6'
                                                 ]].max(axis=1) 

In [59]:
df_mom_2['Max_dpd_Last_3m'] = df_mom_2.loc[:,['dpd_lag_1',
                                                  'dpd_lag_2',
                                                   'dpd_lag_3'
                                                 ]].max(axis=1) 

# Times late payment last 12/9/6/3 mths 

In [60]:
df_mom_2['Times_late_paymt_12mths'] = df_mom_2.loc[:,['late_paymt_flag_lag_1',
                                                  'late_paymt_flag_lag_2',
                                                   'late_paymt_flag_lag_3',
                                                   'late_paymt_flag_lag_4',
                                                   'late_paymt_flag_lag_5',
                                                   'late_paymt_flag_lag_6',
                                                   'late_paymt_flag_lag_7',
                                                   'late_paymt_flag_lag_8',
                                                   'late_paymt_flag_lag_9',
                                                   'late_paymt_flag_lag_10',
                                                    'late_paymt_flag_lag_11',
                                                    'late_paymt_flag_lag_12'
                                                 ]].sum(axis=1) 

In [61]:
df_mom_2['Times_late_paymt_9mths'] = df_mom_2.loc[:,['late_paymt_flag_lag_1',
                                                  'late_paymt_flag_lag_2',
                                                   'late_paymt_flag_lag_3',
                                                   'late_paymt_flag_lag_4',
                                                   'late_paymt_flag_lag_5',
                                                   'late_paymt_flag_lag_6',
                                                   'late_paymt_flag_lag_7',
                                                   'late_paymt_flag_lag_8',
                                                   'late_paymt_flag_lag_9'
                                                 ]].sum(axis=1) 

In [62]:
df_mom_2['Times_late_paymt_6mths'] = df_mom_2.loc[:,['late_paymt_flag_lag_1',
                                                  'late_paymt_flag_lag_2',
                                                   'late_paymt_flag_lag_3',
                                                   'late_paymt_flag_lag_4',
                                                   'late_paymt_flag_lag_5',
                                                   'late_paymt_flag_lag_6'
                                                   ]].sum(axis=1) 

In [63]:
df_mom_2['Times_late_paymt_3mths'] = df_mom_2.loc[:,['late_paymt_flag_lag_1',
                                                  'late_paymt_flag_lag_2',
                                                   'late_paymt_flag_lag_3'
                                                   ]].sum(axis=1) 

### Tenor and balance related variables and ratios


In [64]:
## keeping the negative values also if any
df_mom_2['remaining_tenor'] = round(((df_mom_2["END_DATE"] - df_mom_2["DT_BUSINESSDATE"])/np.timedelta64(1, 'M')))

In [65]:
df_mom_2['paid_down_bal_ratio'] = 1 - (df_mom_2['F_FUTURE_PRINCIPAL']/df_mom_2['LOAN_AMOUNT'])

In [66]:
df_mom_2['tenor_completed'] =  round(((df_mom_2["DT_BUSINESSDATE"] - df_mom_2["START_DATE"])/np.timedelta64(1, 'M')))

In [67]:
df_mom_2['paymnts_made'] = df_mom_2['tenor_completed'] - df_mom_2["I_NO_OF_OVERDUE_EMI"]

#### create MOB variables


In [68]:
df_mom_2["MOB"] =  round(((df_mom_2["DT_BUSINESSDATE"] - df_mom_2["FIRST_DISB_DATE"])/np.timedelta64(1, 'M')))

#### Delq bucket creation

In [69]:
df_mom_2['DEL_BUCKET'] = np.NaN

df_mom_2.loc[df_mom_2['I_DPD'] == 0, 'DEL_BUCKET'] = '1.Current'
df_mom_2.loc[(df_mom_2['I_DPD'] >=1) & (df_mom_2['I_DPD'] <=29) , 'DEL_BUCKET'] = '2.[1-29]'
df_mom_2.loc[(df_mom_2['I_DPD'] >=30) & (df_mom_2['I_DPD'] <=59) , 'DEL_BUCKET'] = '3.[30-59]'
df_mom_2.loc[(df_mom_2['I_DPD'] >=60) & (df_mom_2['I_DPD'] <=89) , 'DEL_BUCKET'] = '4.[60-89]'
df_mom_2.loc[(df_mom_2['I_DPD'] >=90)  , 'DEL_BUCKET'] = '5.[90+)'

In [70]:
df_mom_2['DEL_BUCKET'].value_counts()

1.Current    133408
5.[90+)        6126
4.[60-89]      5664
2.[1-29]       4979
3.[30-59]      3394
Name: DEL_BUCKET, dtype: int64

#### Filter and retain only current (DPD=0) for snapshots

In [71]:
df_mom_0_30 = df_mom_2[(df_mom_2['DEL_BUCKET'] == '1.Current')]

### Filter for only accounts with 6 MOB or more as of snapshot 

In [72]:
df_mom_6MOB = df_mom_0_30[(df_mom_0_30["MOB"] >=min_perf_AO_snapshot) & (df_mom_0_30["MOB"].notnull())]

#### Retain only Required Snapshots for analysis

In [73]:
df_mom_4_snaps = df_mom_6MOB[df_mom_6MOB['DT_BUSINESSDATE'].isin(bus_dates)]

In [74]:
df_mom_4_snaps['DT_BUSINESSDATE'].value_counts()

2019-12-31    3600
2019-09-30    3497
2019-10-31    3398
2019-07-31    3121
Name: DT_BUSINESSDATE, dtype: int64

# Time since last delinquency 

In [75]:
df_temp_ss = df_mom_4_snaps.loc[:,["SZ_LOAN_ACCOUNT_NO","DT_BUSINESSDATE"]]

In [76]:
df_temp_ss = df_temp_ss.rename(columns = {"DT_BUSINESSDATE":"Snapshot_dt"})

In [77]:
df_mom_temp = df_mom_2.loc[:,["SZ_LOAN_ACCOUNT_NO","DT_BUSINESSDATE","I_DPD","flag_30p","flag_60p","flag_90p"]]

In [78]:
df_ss_mom_1 = pd.merge(df_temp_ss,df_mom_temp,how = "left",on="SZ_LOAN_ACCOUNT_NO")

In [79]:
df_ss_mom_2 = df_ss_mom_1[df_ss_mom_1["DT_BUSINESSDATE"]<df_ss_mom_1["Snapshot_dt"]]

In [80]:
df_ss_mom_2["Mths_since_delq_30p"] = np.where(df_ss_mom_2["flag_30p"]==1,
                                         round(((df_ss_mom_2["Snapshot_dt"] - df_ss_mom_2["DT_BUSINESSDATE"])/np.timedelta64(1, 'M'))),np.nan)
df_ss_mom_2["Mths_since_delq_60p"] = np.where(df_ss_mom_2["flag_60p"]==1,
                                         round(((df_ss_mom_2["Snapshot_dt"] - df_ss_mom_2["DT_BUSINESSDATE"])/np.timedelta64(1, 'M'))),np.nan)
df_ss_mom_2["Mths_since_delq_90p"] = np.where(df_ss_mom_2["flag_90p"]==1,
                                         round(((df_ss_mom_2["Snapshot_dt"] - df_ss_mom_2["DT_BUSINESSDATE"])/np.timedelta64(1, 'M'))),np.nan)

In [81]:
Mths_since_delq_30p = df_ss_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO','Snapshot_dt']).agg(mths_since_30p = ("Mths_since_delq_30p","min"))
Mths_since_delq_60p = df_ss_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO','Snapshot_dt']).agg(mths_since_60p = ("Mths_since_delq_60p","min"))
Mths_since_delq_90p = df_ss_mom_2.groupby(['SZ_LOAN_ACCOUNT_NO','Snapshot_dt']).agg(mths_since_90p = ("Mths_since_delq_90p","min"))

In [82]:
Mths_since_delq_30p.reset_index(inplace=True)
Mths_since_delq_60p.reset_index(inplace=True)
Mths_since_delq_90p.reset_index(inplace=True)

In [83]:
since_delq_ds = [Mths_since_delq_30p,Mths_since_delq_60p,Mths_since_delq_90p]

In [84]:
since_delq_ds_all = reduce(lambda  left,right: pd.merge(left,right,on=['SZ_LOAN_ACCOUNT_NO','Snapshot_dt'],
                                            how='outer'), since_delq_ds)

### Bounce data addition

In [85]:
bounce_data = pd.read_excel('D://Varthana/Bounce_data.xlsx')

In [86]:
dwb_col = bounce_data.columns.str.replace('\s+', '_') 

In [87]:
bounce_data.columns = dwb_col

In [88]:
bounce_data['Date'] = pd.to_datetime(bounce_data['month'])

In [89]:
bounce_data['Date_2'] = bounce_data['Date'].dt.to_period('M').dt.to_timestamp('M')

In [90]:
bounce_data['bounce_flag'] = np.where(bounce_data['Status'] == 'Payment Failure',1,0)

In [91]:
bounce_data['bounce_flag'].value_counts()/bounce_data['bounce_flag'].count()

0    0.763492
1    0.236508
Name: bounce_flag, dtype: float64

In [92]:
bounce_data = bounce_data.sort_values(["Contract_Code","Date_2"],ascending=[True,True] )

In [93]:
bounce_data['bounce_flag_lag_1'] = bounce_data.groupby(['Contract_Code'])['bounce_flag'].shift(1)
bounce_data['bounce_flag_lag_2'] = bounce_data.groupby(['Contract_Code'])['bounce_flag'].shift(2)
bounce_data['bounce_flag_lag_3'] = bounce_data.groupby(['Contract_Code'])['bounce_flag'].shift(3)
bounce_data['bounce_flag_lag_4'] = bounce_data.groupby(['Contract_Code'])['bounce_flag'].shift(4)
bounce_data['bounce_flag_lag_5'] = bounce_data.groupby(['Contract_Code'])['bounce_flag'].shift(5)
bounce_data['bounce_flag_lag_6'] = bounce_data.groupby(['Contract_Code'])['bounce_flag'].shift(6)
bounce_data['bounce_flag_lag_7'] = bounce_data.groupby(['Contract_Code'])['bounce_flag'].shift(7)
bounce_data['bounce_flag_lag_8'] = bounce_data.groupby(['Contract_Code'])['bounce_flag'].shift(8)
bounce_data['bounce_flag_lag_9'] = bounce_data.groupby(['Contract_Code'])['bounce_flag'].shift(9)
bounce_data['bounce_flag_lag_10'] = bounce_data.groupby(['Contract_Code'])['bounce_flag'].shift(10)
bounce_data['bounce_flag_lag_11'] = bounce_data.groupby(['Contract_Code'])['bounce_flag'].shift(11)
bounce_data['bounce_flag_lag_12'] = bounce_data.groupby(['Contract_Code'])['bounce_flag'].shift(12)

In [94]:
bounce_data_2 = pd.merge(bounce_data,LMS_det_all_1,left_on=['Contract_Code'],
                         right_on=['LOAN_ID'],how='left',)

In [95]:
bounce_data_3 = bounce_data_2.drop(columns = ['Branch_Name',
                                             'School_Name',
                                             'Status',
                                             'month',
                                             'Date',
                                              'TENOR',
                                              'END_DATE',
                                              'LOAN_AMOUNT'
                                             ],axis=1)

In [96]:
bounce_data_3['Num_bounce_L3M'] = bounce_data_3.loc[:,['bounce_flag_lag_1',
                                                       'bounce_flag_lag_2',
                                                       'bounce_flag_lag_3']].sum(axis=1)

In [97]:
bounce_data_3['Num_bounce_L6M'] = bounce_data_3.loc[:,['bounce_flag_lag_1',
                                                       'bounce_flag_lag_2',
                                                       'bounce_flag_lag_3',
                                                       'bounce_flag_lag_4',
                                                       'bounce_flag_lag_5',
                                                       'bounce_flag_lag_6']].sum(axis=1)

In [98]:
bounce_data_3['Num_bounce_L9M'] = bounce_data_3.loc[:,['bounce_flag_lag_1',
                                                        'bounce_flag_lag_2',
                                                        'bounce_flag_lag_3',
                                                        'bounce_flag_lag_4',
                                                        'bounce_flag_lag_5',
                                                        'bounce_flag_lag_6',
                                                        'bounce_flag_lag_7',
                                                        'bounce_flag_lag_8',
                                                        'bounce_flag_lag_9']].sum(axis=1)

In [99]:
bounce_data_3['Num_bounce_L12M'] = bounce_data_3.loc[:,['bounce_flag_lag_1',
                                                       'bounce_flag_lag_2',
                                                       'bounce_flag_lag_3',
                                                       'bounce_flag_lag_4',
                                                       'bounce_flag_lag_5',
                                                       'bounce_flag_lag_6',
                                                       'bounce_flag_lag_7',
                                                       'bounce_flag_lag_8',
                                                       'bounce_flag_lag_9',
                                                       'bounce_flag_lag_10',
                                                       'bounce_flag_lag_11',
                                                       'bounce_flag_lag_12'
                                                       ]].sum(axis=1)

In [100]:
bounce_data_3 = bounce_data_3.sort_values(["Contract_Code","Date_2"],ascending=[True,True] )

In [101]:
bounce_data_3['Total_bounces'] = bounce_data_3.groupby(['Contract_Code'])['bounce_flag'].apply(lambda x: x.cumsum())

In [102]:
bounce_data_3['MOB'] = round(((bounce_data_3["Date_2"] - bounce_data_3["FIRST_DISB_DATE"])/np.timedelta64(1, 'M')))

In [103]:
bounce_data_3['perc_bounces'] = bounce_data_3['Total_bounces']/bounce_data_3['MOB']

In [104]:
final_bounce_dat = pd.DataFrame(bounce_data_3[['Contract_Code',
                                'Date_2',
                                'Num_bounce_L3M',
                                'Num_bounce_L6M',
                                'Num_bounce_L9M',
                                'Num_bounce_L12M',
                                'Total_bounces',
                                 'perc_bounces'
                                ]])

In [105]:
final_bounce_dat = final_bounce_dat.rename(columns = {"Contract_Code":"SZ_LOAN_ACCOUNT_NO",
                                                     "Date_2":"DT_BUSINESSDATE"})

#### Base accounts Aggregation

In [106]:
df_mom_4_snaps_2 = pd.merge(df_mom_4_snaps,since_delq_ds_all, 
                            left_on = ['SZ_LOAN_ACCOUNT_NO','DT_BUSINESSDATE'],
                            right_on = ['SZ_LOAN_ACCOUNT_NO','Snapshot_dt'], how="left")

In [107]:
df_mom_4_snaps_3  = pd.merge(df_mom_4_snaps_2,final_bounce_dat,
                            on = ['SZ_LOAN_ACCOUNT_NO','DT_BUSINESSDATE'],
                            how="left")

### Acq - time independent variables additions

In [108]:
schl_det = pd.read_csv("D://Varthana/school_details_updated.csv", low_memory =  False)

In [109]:
schl_det['SCH_BLD_OWN_FLAG'] = np.where(schl_det['SCH_BUILDING'] == 'OWNED',1,0)

In [110]:
schl_det_1 =  schl_det.loc[:,['SZ_APPLICATION_NO','SZ_LOCALITY','SCH_BLD_OWN_FLAG']]

In [111]:
schl_det_1.rename(columns = {'SZ_APPLICATION_NO':'SZ_LOAN_ACCOUNT_NO',
                             'SZ_LOCALITY':'school_locality'}, inplace =  True)

In [112]:
collat_det = pd.read_csv("D://Varthana/COLLATERAL_DETAILS_07-Sep-20.csv", low_memory =  False)

In [113]:
collat_det_1 = collat_det.loc[:,['SZ_APPLICATION_NO','TYPE']]

In [114]:
collat_det_1.rename(columns = {'SZ_APPLICATION_NO':'SZ_LOAN_ACCOUNT_NO',
                                'TYPE':'collat_type'}, inplace=True)

In [115]:
collat_det_1['LAND']= np.where(collat_det_1['collat_type'] == 'L', 1,0)
collat_det_1['BUILDING']= np.where(collat_det_1['collat_type'] == 'B', 1,0)

In [116]:
collat_det_2 = collat_det_1.groupby(['SZ_LOAN_ACCOUNT_NO']).agg(num_land =  ("LAND","sum"),
                                                              num_build = ("BUILDING","sum")).reset_index()


In [117]:
collat_det_2['land_flag'] = np.where(collat_det_2['num_land'] >=1,1,0)
collat_det_2['building_flag'] = np.where(collat_det_2['num_build'] >=1,1,0)
collat_det_2['both_L_B_flag'] = np.where((collat_det_2['num_build'] >=1) & (collat_det_2['num_land'] >=1),1,0)

In [118]:
df_mom_4_snaps_4 = pd.merge(df_mom_4_snaps_3,schl_det_1, on='SZ_LOAN_ACCOUNT_NO', how='left')

In [119]:
df_mom_4_snaps_5 = pd.merge(df_mom_4_snaps_4,collat_det_2, on='SZ_LOAN_ACCOUNT_NO', how='left')

### LTV Calculation

In [120]:
Loan_det = pd.read_csv("D://Varthana/LOS_LOAN_DETAILS_07-Sep-20.csv", low_memory =  False)

In [121]:
Loan_det.rename(columns = {"APPLICATION_NO":"SZ_LOAN_ACCOUNT_NO"},inplace =  True)

In [122]:
df_mom_4_snaps_6 = pd.merge(df_mom_4_snaps_5,Loan_det.loc[:,['SZ_LOAN_ACCOUNT_NO','TOTAL_COLLATERAL_VALUE']],
                           how = "left", on = "SZ_LOAN_ACCOUNT_NO")

In [123]:
df_mom_4_snaps_6["LTV"] = df_mom_4_snaps_6["LOAN_AMOUNT"]/df_mom_4_snaps_6["TOTAL_COLLATERAL_VALUE"]

## Bad flag addition - x days ever in N3M

In [124]:
x_days_ever_N3M = pd.read_csv('D://Varthana/xdays_ever_N3M_dv_trigMod.csv')

In [125]:
x_days_ever_N3M['Snapshot_dt'] = pd.to_datetime(x_days_ever_N3M['Snapshot_dt'])

In [126]:
### these accounts dont have performance post the corresponding snapshot.. 
# i.e., Snapshot is the last business data available in the MoM data 

diff_list = np.setdiff1d(list(df_mom_4_snaps_6['SZ_LOAN_ACCOUNT_NO'][df_mom_4_snaps_6['Snapshot_dt']=='2019-09-30']),
                         list(x_days_ever_N3M['SZ_LOAN_ACCOUNT_NO'][(x_days_ever_N3M['MOB']>=6)&
                            (x_days_ever_N3M['DEL_BUCKET']=='1.Current')&
                           (x_days_ever_N3M['Snapshot_dt']=='2019-09-30')]))

###### Filtering only accounts with >=6MOB and current accounts from Perforamnce data to Inner join

In [127]:
x_days_ever_N3M_2 = x_days_ever_N3M[(x_days_ever_N3M['MOB']>=6)&(x_days_ever_N3M['DEL_BUCKET']=='1.Current')]

In [128]:
df_mom_w_perf = pd.merge(df_mom_4_snaps_6, x_days_ever_N3M_2.loc[:,["SZ_LOAN_ACCOUNT_NO","Snapshot_dt","xdays_ever_3m"]],
                         how='inner', on = ['SZ_LOAN_ACCOUNT_NO','Snapshot_dt'])

In [129]:
df_mom_w_perf['Snapshot_dt'].value_counts()

2019-12-31    3533
2019-09-30    3408
2019-10-31    3322
2019-07-31    3056
Name: Snapshot_dt, dtype: int64

In [130]:
df_mom_w_perf['xdays_ever_3m'].value_counts()/df_mom_w_perf['xdays_ever_3m'].count()

0    0.93498
1    0.06502
Name: xdays_ever_3m, dtype: float64

## Key creation with LAN & Date 

In [131]:
df_mom_w_perf['Appl_datekey'] = df_mom_w_perf["SZ_LOAN_ACCOUNT_NO"].str.cat(df_mom_w_perf["Snapshot_dt"].astype(str), sep="_")

## Retain only required variables


In [132]:
df_mom_w_perf_pick = pd.DataFrame(df_mom_w_perf[['SZ_LOAN_ACCOUNT_NO',
'Appl_datekey',
'Snapshot_dt',
'MOB',
'DEL_BUCKET',
'I_NO_OF_PAID_EMI',
'I_NO_OF_OVERDUE_EMI',
'I_NO_OF_UNPAID_EMI',
'TENOR',
'LOAN_AMOUNT',
'days_late_paid',
'late_paymt_flag',
'Times_30p_12m',
'Times_60p_12m',
'Times_90p_12m',
'Times_xdays_12m',
'Times_30p_9m',
'Times_60p_9m',
'Times_90p_9m',
'Times_xdays_9m',
'Times_30p_6m',
'Times_60p_6m',
'Times_90p_6m',
'Times_xdays_6m',
'Times_30p_3m',
'Times_60p_3m',
'Times_90p_3m',
'Times_xdays_3m',
'Last_12mths_30p_flag',
'Last_12mths_60p_flag',
'Last_12mths_90p_flag',
'Last_12mths_xdays_flag',
'Last_9mths_30p_flag',
'Last_9mths_60p_flag',
'Last_9mths_90p_flag',
'Last_9mths_xdays_flag',
'Last_6mths_30p_flag',
'Last_6mths_60p_flag',
'Last_6mths_90p_flag',
'Last_6mths_xdays_flag',
'Last_3mths_30p_flag',
'Last_3mths_60p_flag',
'Last_3mths_90p_flag',
'Last_3mths_xdays_flag',
'Max_dpd_Last_12m',
'Max_dpd_Last_9m',
'Max_dpd_Last_6m',
'Max_dpd_Last_3m',
'Times_late_paymt_12mths',
'Times_late_paymt_9mths',
'Times_late_paymt_6mths',
'Times_late_paymt_3mths',
'remaining_tenor',
'paid_down_bal_ratio',
'tenor_completed',
'paymnts_made',
'mths_since_30p',
'mths_since_60p',
'mths_since_90p',
'Num_bounce_L3M',
'Num_bounce_L6M',
'Num_bounce_L9M',
'Num_bounce_L12M',
'Total_bounces',
'perc_bounces',
'school_locality',
'SCH_BLD_OWN_FLAG',
'num_land',
'num_build',
'land_flag',
'building_flag',
'both_L_B_flag',
'LTV',
'xdays_ever_3m']]
)


In [133]:
df_mom_w_perf_pick[df_mom_w_perf_pick['days_late_paid']<0].loc[:,["SZ_LOAN_ACCOUNT_NO","days_late_paid"]]

,SZ_LOAN_ACCOUNT_NO,days_late_paid
680,BLR07989U,-30.0
681,BLR07989U,-27.0
682,BLR07989U,-27.0
683,BLR07989U,-61.0
2857,JAI2090S,-676.0
2858,JAI2090S,-738.0
2859,JAI2090S,-768.0
2860,JAI2090S,-829.0


### Model ready internal data 

In [134]:
df_mom_w_perf_pick['Snapshot_dt'].value_counts()

2019-12-31    3533
2019-09-30    3408
2019-10-31    3322
2019-07-31    3056
Name: Snapshot_dt, dtype: int64

In [135]:
df_mom_w_perf_jul_oct_dec19 = df_mom_w_perf_pick[(df_mom_w_perf_pick['Snapshot_dt']=='2019-07-31') |
                                            (df_mom_w_perf_pick['Snapshot_dt']=='2019-10-31') |
                                            (df_mom_w_perf_pick['Snapshot_dt']=='2019-12-31')]

###### Imputing nans & inf with spl values 

In [136]:
df_mom_w_perf_jul_oct_dec19.fillna(-99999, inplace=True)
df_mom_w_perf_jul_oct_dec19.replace(np.inf, -99999, inplace=True)

##### internal data export 

In [137]:
#df_mom_w_perf_jul_oct_dec19[(df_mom_w_perf_pick['Snapshot_dt']=='2019-07-31') |
#                        (df_mom_w_perf_pick['Snapshot_dt']=='2019-10-31') ].drop(['SZ_LOAN_ACCOUNT_NO','Snapshot_dt','DEL_BUCKET'],axis=1).to_csv('/home/siddarth/LMS_data/Input_CSV/internal_data_jul_oct19.csv',index=False)

### Split the data for buruea merge 

In [138]:
df_mom_w_perf_jul19 = df_mom_w_perf_jul_oct_dec19[df_mom_w_perf_jul_oct_dec19['Snapshot_dt']=='2019-07-31']

In [139]:
df_mom_w_perf_oct19 = df_mom_w_perf_jul_oct_dec19[df_mom_w_perf_jul_oct_dec19['Snapshot_dt']=='2019-10-31']

In [140]:
df_mom_w_perf_dec19 = df_mom_w_perf_jul_oct_dec19[df_mom_w_perf_jul_oct_dec19['Snapshot_dt']=='2019-12-31']

In [141]:
df_mom_w_perf_dec19.shape, df_mom_w_perf_oct19.shape,df_mom_w_perf_jul19.shape

((3533, 74), (3322, 74), (3056, 74))

In [142]:
df_mom_w_perf_jul19.to_csv('D://Varthana/internal_data_jul19.csv',index=False)
df_mom_w_perf_oct19.to_csv('D://Varthana/internal_data_oct19.csv',index=False)
df_mom_w_perf_dec19.to_csv('D://Varthana/internal_data_dec19.csv',index=False)

### Bureau roll up

In [143]:
from datetime import datetime
import time
import datetime
import dateutil.relativedelta

In [144]:
bur_q1=pd.read_csv('D:\Varthana\Raw Scrub data/Raw Scrub data - Q1.csv')
bur_q2=pd.read_csv('D:\Varthana\Raw Scrub data/Raw Scrub data - Q2.csv')
bur_q3=pd.read_csv('D:\Varthana\Raw Scrub data/Raw Scrub data - Q3.csv')
# print(df.shape)

In [145]:
def bur_rollup(df):
    df['MemberReference_Orig'] = df['MemberReference']
    df['MemberReference'] = df['MemberReference'].str.replace(r'-XAC', '')
    ## Imputing Payment history
    df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']] = df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']].replace('XXX', 0)
    df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']] = df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']].replace('STD', 1)
    df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']] = df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']].replace('SMA', 1)
    df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']] = df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']].replace('SUB', 90)
    df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']] = df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']].replace('DBT', 90)
    df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']] = df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']].replace('LSS', 90)
    df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']] = df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']].replace('   ', 0)
    
    cols = ['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12','Paymt_hst_13','Paymt_hst_14','Paymt_hst_15','Paymt_hst_16','Paymt_hst_17','Paymt_hst_18','Paymt_hst_19','Paymt_hst_20','Paymt_hst_21','Paymt_hst_22','Paymt_hst_23','Paymt_hst_24','Paymt_hst_25','Paymt_hst_26','Paymt_hst_27','Paymt_hst_28','Paymt_hst_29','Paymt_hst_30','Paymt_hst_31','Paymt_hst_32','Paymt_hst_33','Paymt_hst_34','Paymt_hst_35','Paymt_hst_36']
    df[cols] = df[cols].astype(int)
    
    ## Times xp in 3/6/9/12 mths
    df['num_times_Xp_L3M'] = np.where(((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01']<=29))|((df['Paymt_hst_02']>=1) & (df['Paymt_hst_02']<=29))|((df['Paymt_hst_03']>=1) &(df['Paymt_hst_03']<=29)),(df['Paymt_hst_01']<=29,df['Paymt_hst_02']<=29,df['Paymt_hst_03']<=29),0).sum(axis=0)
    df['num_times_Xp_L6M'] = np.where(((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01']<=29))|((df['Paymt_hst_02']>=1) & (df['Paymt_hst_02']<=29))|((df['Paymt_hst_03']>=1) &(df['Paymt_hst_03']<=29))|((df['Paymt_hst_04']>=1) & (df['Paymt_hst_04']<=29))|((df['Paymt_hst_05']>=1) & (df['Paymt_hst_05']<=29))|((df['Paymt_hst_06']>=1) &(df['Paymt_hst_06']<=29)),(df['Paymt_hst_01']<=29,df['Paymt_hst_02']<=29,df['Paymt_hst_03']<=29,df['Paymt_hst_04']<=29,df['Paymt_hst_05']<=29,df['Paymt_hst_06']<=29),0).sum(axis=0)
    df['num_times_Xp_L9M'] = np.where(((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01']<=29))|((df['Paymt_hst_02']>=1) & (df['Paymt_hst_02']<=29))|((df['Paymt_hst_03']>=1) &(df['Paymt_hst_03']<=29))|((df['Paymt_hst_04']>=1) & (df['Paymt_hst_04']<=29))|((df['Paymt_hst_05']>=1) & (df['Paymt_hst_05']<=29))|((df['Paymt_hst_06']>=1) &(df['Paymt_hst_06']<=29))|((df['Paymt_hst_07']>=1) & (df['Paymt_hst_07']<=29))|((df['Paymt_hst_08']>=1) & (df['Paymt_hst_08']<=29))|((df['Paymt_hst_09']>=1) &(df['Paymt_hst_09']<=29)),(df['Paymt_hst_01']<=29,df['Paymt_hst_02']<=29,df['Paymt_hst_03']<=29,df['Paymt_hst_04']<=29,df['Paymt_hst_05']<=29,df['Paymt_hst_06']<=29,df['Paymt_hst_07']<=29,df['Paymt_hst_08']<=29,df['Paymt_hst_09']<=29),0).sum(axis=0)
    df['num_times_Xp_L12M'] = np.where(((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01']<=29))|((df['Paymt_hst_02']>=1) & (df['Paymt_hst_02']<=29))|((df['Paymt_hst_03']>=1) &(df['Paymt_hst_03']<=29))|((df['Paymt_hst_04']>=1) & (df['Paymt_hst_04']<=29))|((df['Paymt_hst_05']>=1) & (df['Paymt_hst_05']<=29))|((df['Paymt_hst_06']>=1) &(df['Paymt_hst_06']<=29))|((df['Paymt_hst_07']>=1) & (df['Paymt_hst_07']<=29))|((df['Paymt_hst_08']>=1) & (df['Paymt_hst_08']<=29))|((df['Paymt_hst_09']>=1) &(df['Paymt_hst_09']<=29))|((df['Paymt_hst_10']>=1) & (df['Paymt_hst_10']<=29))|((df['Paymt_hst_11']>=1) & (df['Paymt_hst_11']<=29))|((df['Paymt_hst_12']>=1) &(df['Paymt_hst_12']<=29)),(df['Paymt_hst_01']<=29,df['Paymt_hst_02']<=29,df['Paymt_hst_03']<=29,df['Paymt_hst_04']<=29,df['Paymt_hst_05']<=29,df['Paymt_hst_06']<=29,df['Paymt_hst_07']<=29,df['Paymt_hst_08']<=29,df['Paymt_hst_09']<=29,df['Paymt_hst_10']<=29,df['Paymt_hst_11']<=29,df['Paymt_hst_12']<=29),0).sum(axis=0)
    
    
    ## Times 30p in 3/6/9/12 mths
    df['num_times_30p_L3M'] = np.where((df['Paymt_hst_01']>=30)|(df['Paymt_hst_02']>=30)|(df['Paymt_hst_03']>=30),(df['Paymt_hst_01']>=30,df['Paymt_hst_02']>=30,df['Paymt_hst_03']>=30),0).sum(axis=0)
    df['num_times_30p_L6M'] = np.where((df['Paymt_hst_01'] >= 30) | (df['Paymt_hst_02'] >= 30) | (df['Paymt_hst_03'] >= 30) | (df['Paymt_hst_04'] >= 30) | (df['Paymt_hst_05'] >= 30) | (df['Paymt_hst_06'] >= 30),(df['Paymt_hst_01'] >= 30,df['Paymt_hst_02'] >= 30,df['Paymt_hst_03'] >= 30,df['Paymt_hst_04'] >= 30,df['Paymt_hst_05'] >= 30,df['Paymt_hst_06'] >= 30),0).sum(axis=0)
    df['num_times_30p_L9M'] = np.where((df['Paymt_hst_01'] >= 30) | (df['Paymt_hst_02'] >= 30) | (df['Paymt_hst_03'] >= 30) | (df['Paymt_hst_04'] >= 30) | (df['Paymt_hst_05'] >= 30) | (df['Paymt_hst_06'] >= 30)|(df['Paymt_hst_07'] >= 30) | (df['Paymt_hst_08'] >= 30) | (df['Paymt_hst_09'] >= 30),(df['Paymt_hst_01'] >= 30,df['Paymt_hst_02'] >= 30,df['Paymt_hst_03'] >= 30,df['Paymt_hst_04'] >= 30,df['Paymt_hst_05'] >= 30,df['Paymt_hst_06'] >= 30,df['Paymt_hst_07'] >= 30,df['Paymt_hst_08'] >= 30,df['Paymt_hst_09'] >= 30),0).sum(axis=0)
    df['num_times_30p_L12M'] = np.where((df['Paymt_hst_01'] >= 30) | (df['Paymt_hst_02'] >= 30) | (df['Paymt_hst_03'] >= 30) | (df['Paymt_hst_04'] >= 30) | (df['Paymt_hst_05'] >= 30) | (df['Paymt_hst_06'] >= 30)|(df['Paymt_hst_07'] >= 30) | (df['Paymt_hst_08'] >= 30) | (df['Paymt_hst_09'] >= 30) | (df['Paymt_hst_10'] >= 30) | (df['Paymt_hst_11'] >= 30) | (df['Paymt_hst_12'] >= 30),(df['Paymt_hst_01'] >= 30,df['Paymt_hst_02'] >= 30,df['Paymt_hst_03'] >= 30,df['Paymt_hst_04'] >= 30,df['Paymt_hst_05'] >= 30,df['Paymt_hst_06'] >= 30,df['Paymt_hst_07'] >= 30,df['Paymt_hst_08'] >= 30,df['Paymt_hst_09'] >= 30,df['Paymt_hst_10'] >= 30,df['Paymt_hst_11'] >= 30,df['Paymt_hst_12'] >= 30),0).sum(axis=0)
    
    ## Times 60p in 3/6/9/12 mths
    df['num_times_60p_L3M'] = np.where((df['Paymt_hst_01']>=60)|(df['Paymt_hst_02']>=60)|(df['Paymt_hst_03']>=60),(df['Paymt_hst_01']>=60,df['Paymt_hst_02']>=60,df['Paymt_hst_03']>=60),0).sum(axis=0)
    df['num_times_60p_L6M'] = np.where((df['Paymt_hst_01'] >= 60) | (df['Paymt_hst_02'] >= 60) | (df['Paymt_hst_03'] >= 60) | (df['Paymt_hst_04'] >= 60) | (df['Paymt_hst_05'] >= 60) | (df['Paymt_hst_06'] >= 60),(df['Paymt_hst_01'] >= 60,df['Paymt_hst_02'] >= 60,df['Paymt_hst_03'] >= 60,df['Paymt_hst_04'] >= 60,df['Paymt_hst_05'] >= 60,df['Paymt_hst_06'] >= 60),0).sum(axis=0)
    df['num_times_60p_L9M'] = np.where((df['Paymt_hst_01'] >= 60) | (df['Paymt_hst_02'] >= 60) | (df['Paymt_hst_03'] >= 60) | (df['Paymt_hst_04'] >= 60) | (df['Paymt_hst_05'] >= 60) | (df['Paymt_hst_06'] >= 60)|(df['Paymt_hst_07'] >= 60) | (df['Paymt_hst_08'] >= 60) | (df['Paymt_hst_09'] >= 60),(df['Paymt_hst_01'] >= 60,df['Paymt_hst_02'] >= 60,df['Paymt_hst_03'] >= 60,df['Paymt_hst_04'] >= 60,df['Paymt_hst_05'] >= 60,df['Paymt_hst_06'] >= 60,df['Paymt_hst_07'] >= 60,df['Paymt_hst_08'] >= 60,df['Paymt_hst_09'] >= 60),0).sum(axis=0)
    df['num_times_60p_L12M'] = np.where((df['Paymt_hst_01'] >= 60) | (df['Paymt_hst_02'] >= 60) | (df['Paymt_hst_03'] >= 60) | (df['Paymt_hst_04'] >= 60) | (df['Paymt_hst_05'] >= 60) | (df['Paymt_hst_06'] >= 60)|(df['Paymt_hst_07'] >= 60) | (df['Paymt_hst_08'] >= 60) | (df['Paymt_hst_09'] >= 60) | (df['Paymt_hst_10'] >= 60) | (df['Paymt_hst_11'] >= 60) | (df['Paymt_hst_12'] >= 60),(df['Paymt_hst_01'] >= 60,df['Paymt_hst_02'] >= 60,df['Paymt_hst_03'] >= 60,df['Paymt_hst_04'] >= 60,df['Paymt_hst_05'] >= 60,df['Paymt_hst_06'] >= 60,df['Paymt_hst_07'] >= 60,df['Paymt_hst_08'] >= 60,df['Paymt_hst_09'] >= 60,df['Paymt_hst_10'] >= 60,df['Paymt_hst_11'] >= 60,df['Paymt_hst_12'] >= 60),0).sum(axis=0)
    
    ## Times 90p in 3/6/9/12 mths
    df['num_times_90p_L3M'] = np.where((df['Paymt_hst_01']>=90)|(df['Paymt_hst_02']>=90)|(df['Paymt_hst_03']>=90),(df['Paymt_hst_01']>=90,df['Paymt_hst_02']>=90,df['Paymt_hst_03']>=90),0).sum(axis=0)
    df['num_times_90p_L6M'] = np.where((df['Paymt_hst_01'] >= 90) | (df['Paymt_hst_02'] >= 90) | (df['Paymt_hst_03'] >= 90) | (df['Paymt_hst_04'] >= 90) | (df['Paymt_hst_05'] >= 90) | (df['Paymt_hst_06'] >= 90),(df['Paymt_hst_01'] >= 90,df['Paymt_hst_02'] >= 90,df['Paymt_hst_03'] >= 90,df['Paymt_hst_04'] >= 90,df['Paymt_hst_05'] >= 90,df['Paymt_hst_06'] >= 90),0).sum(axis=0)
    df['num_times_90p_L9M'] = np.where((df['Paymt_hst_01'] >= 90) | (df['Paymt_hst_02'] >= 90) | (df['Paymt_hst_03'] >= 90) | (df['Paymt_hst_04'] >= 90) | (df['Paymt_hst_05'] >= 90) | (df['Paymt_hst_06'] >= 90)|(df['Paymt_hst_07'] >= 90) | (df['Paymt_hst_08'] >= 90) | (df['Paymt_hst_09'] >= 90),(df['Paymt_hst_01'] >= 90,df['Paymt_hst_02'] >= 90,df['Paymt_hst_03'] >= 90,df['Paymt_hst_04'] >= 90,df['Paymt_hst_05'] >= 90,df['Paymt_hst_06'] >= 90,df['Paymt_hst_07'] >= 90,df['Paymt_hst_08'] >= 90,df['Paymt_hst_09'] >= 90),0).sum(axis=0)
    df['num_times_90p_L12M'] = np.where((df['Paymt_hst_01'] >= 90) | (df['Paymt_hst_02'] >= 90) | (df['Paymt_hst_03'] >= 90) | (df['Paymt_hst_04'] >= 90) | (df['Paymt_hst_05'] >= 90) | (df['Paymt_hst_06'] >= 90)|(df['Paymt_hst_07'] >= 90) | (df['Paymt_hst_08'] >= 90) | (df['Paymt_hst_09'] >= 90) | (df['Paymt_hst_10'] >= 90) | (df['Paymt_hst_11'] >= 90) | (df['Paymt_hst_12'] >= 90),(df['Paymt_hst_01'] >= 90,df['Paymt_hst_02'] >= 90,df['Paymt_hst_03'] >= 90,df['Paymt_hst_04'] >= 90,df['Paymt_hst_05'] >= 90,df['Paymt_hst_06'] >= 90,df['Paymt_hst_07'] >= 90,df['Paymt_hst_08'] >= 90,df['Paymt_hst_09'] >= 90,df['Paymt_hst_10'] >= 90,df['Paymt_hst_11'] >= 90,df['Paymt_hst_12'] >= 90),0).sum(axis=0)
    
    ## flag xp in 3/6/9/12 mths
    df['X_DPD_L3M_Flag'] = np.where(((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01']<=29))|((df['Paymt_hst_02']>=1) & (df['Paymt_hst_02']<=29))|((df['Paymt_hst_03']>=1) &(df['Paymt_hst_03']<=29)),1,0)
    df['X_DPD_L6M_Flag'] = np.where(((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01']<=29))|((df['Paymt_hst_02']>=1) & (df['Paymt_hst_02']<=29))|((df['Paymt_hst_03']>=1) &(df['Paymt_hst_03']<=29))|((df['Paymt_hst_04']>=1) & (df['Paymt_hst_04']<=29))|((df['Paymt_hst_05']>=1) & (df['Paymt_hst_05']<=29))|((df['Paymt_hst_06']>=1) &(df['Paymt_hst_06']<=29)),1,0)
    df['X_DPD_L9M_Flag'] = np.where(((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01']<=29))|((df['Paymt_hst_02']>=1) & (df['Paymt_hst_02']<=29))|((df['Paymt_hst_03']>=1) &(df['Paymt_hst_03']<=29))|((df['Paymt_hst_04']>=1) & (df['Paymt_hst_04']<=29))|((df['Paymt_hst_05']>=1) & (df['Paymt_hst_05']<=29))|((df['Paymt_hst_06']>=1) &(df['Paymt_hst_06']<=29))|((df['Paymt_hst_07']>=1) & (df['Paymt_hst_07']<=29))|((df['Paymt_hst_08']>=1) & (df['Paymt_hst_08']<=29))|((df['Paymt_hst_09']>=1) &(df['Paymt_hst_09']<=29)),1,0)
    df['X_DPD_L12M_Flag'] = np.where(((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01']<=29))|((df['Paymt_hst_02']>=1) & (df['Paymt_hst_02']<=29))|((df['Paymt_hst_03']>=1) &(df['Paymt_hst_03']<=29))|((df['Paymt_hst_04']>=1) & (df['Paymt_hst_04']<=29))|((df['Paymt_hst_05']>=1) & (df['Paymt_hst_05']<=29))|((df['Paymt_hst_06']>=1) &(df['Paymt_hst_06']<=29))|((df['Paymt_hst_07']>=1) & (df['Paymt_hst_07']<=29))|((df['Paymt_hst_08']>=1) & (df['Paymt_hst_08']<=29))|((df['Paymt_hst_09']>=1) &(df['Paymt_hst_09']<=29))|((df['Paymt_hst_10']>=1) & (df['Paymt_hst_10']<=29))|((df['Paymt_hst_11']>=1) & (df['Paymt_hst_11']<=29))|((df['Paymt_hst_12']>=1) &(df['Paymt_hst_12']<=29)),1,0)
    
    ## flag 30p in 3/6/9/12 mths
    df['30p_DPD_L3M_Flag'] = np.where((df['Paymt_hst_01'] > 30) | (df['Paymt_hst_02'] > 30) | (df['Paymt_hst_03'] > 30) ,1,0)
    df['30p_DPD_L6M_Flag'] = np.where((df['Paymt_hst_01'] > 30) | (df['Paymt_hst_02'] > 30) | (df['Paymt_hst_03'] > 30) | (df['Paymt_hst_04'] > 30) | (df['Paymt_hst_05'] > 30) | (df['Paymt_hst_06'] > 30) ,1,0)
    df['30p_DPD_L9M_Flag'] = np.where((df['Paymt_hst_01'] > 30) | (df['Paymt_hst_02'] > 30) | (df['Paymt_hst_03'] > 30) | (df['Paymt_hst_04'] > 30) | (df['Paymt_hst_05'] > 30) | (df['Paymt_hst_06'] > 30)|(df['Paymt_hst_07'] > 30) | (df['Paymt_hst_08'] > 30) | (df['Paymt_hst_09'] > 30) ,1,0)
    df['30p_DPD_L12M_Flag'] = np.where((df['Paymt_hst_01'] > 30) | (df['Paymt_hst_02'] > 30) | (df['Paymt_hst_03'] > 30) | (df['Paymt_hst_04'] > 30) | (df['Paymt_hst_05'] > 30) | (df['Paymt_hst_06'] > 30)|(df['Paymt_hst_07'] > 30) | (df['Paymt_hst_08'] > 30) | (df['Paymt_hst_09'] > 30) | (df['Paymt_hst_10'] > 30) | (df['Paymt_hst_11'] > 30) | (df['Paymt_hst_12'] > 30) ,1,0)
    
    ## flag 60p in 3/6/9/12 mths
    df['60p_DPD_L3M_Flag'] = np.where((df['Paymt_hst_01'] > 60) | (df['Paymt_hst_02'] > 60) | (df['Paymt_hst_03'] > 60) ,1,0)
    df['60p_DPD_L6M_Flag'] = np.where((df['Paymt_hst_01'] > 60) | (df['Paymt_hst_02'] > 60) | (df['Paymt_hst_03'] > 60) | (df['Paymt_hst_04'] > 60) | (df['Paymt_hst_05'] > 60) | (df['Paymt_hst_06'] > 60) ,1,0)
    df['60p_DPD_L9M_Flag'] = np.where((df['Paymt_hst_01'] > 60) | (df['Paymt_hst_02'] > 60) | (df['Paymt_hst_03'] > 60) | (df['Paymt_hst_04'] > 60) | (df['Paymt_hst_05'] > 60) | (df['Paymt_hst_06'] > 60)|(df['Paymt_hst_07'] > 60) | (df['Paymt_hst_08'] > 60) | (df['Paymt_hst_09'] > 60) ,1,0)
    df['60p_DPD_L12M_Flag'] = np.where((df['Paymt_hst_01'] > 60) | (df['Paymt_hst_02'] > 60) | (df['Paymt_hst_03'] > 60) | (df['Paymt_hst_04'] > 60) | (df['Paymt_hst_05'] > 60) | (df['Paymt_hst_06'] > 60)|(df['Paymt_hst_07'] > 60) | (df['Paymt_hst_08'] > 60) | (df['Paymt_hst_09'] > 60) | (df['Paymt_hst_10'] > 60) | (df['Paymt_hst_11'] > 60) | (df['Paymt_hst_12'] > 60) ,1,0)
    
    ## flag 90p in 3/6/9/12 mths
    df['90p_DPD_L3M_Flag'] = np.where((df['Paymt_hst_01'] > 90) | (df['Paymt_hst_02'] > 90) | (df['Paymt_hst_03'] > 90) ,1,0)
    df['90p_DPD_L6M_Flag'] = np.where((df['Paymt_hst_01'] > 90) | (df['Paymt_hst_02'] > 90) | (df['Paymt_hst_03'] > 90) | (df['Paymt_hst_04'] > 90) | (df['Paymt_hst_05'] > 90) | (df['Paymt_hst_06'] > 90) ,1,0)
    df['90p_DPD_L9M_Flag'] = np.where((df['Paymt_hst_01'] > 90) | (df['Paymt_hst_02'] > 90) | (df['Paymt_hst_03'] > 90) | (df['Paymt_hst_04'] > 90) | (df['Paymt_hst_05'] > 90) | (df['Paymt_hst_06'] > 90)|(df['Paymt_hst_07'] > 90) | (df['Paymt_hst_08'] > 90) | (df['Paymt_hst_09'] > 90) ,1,0)
    df['90p_DPD_L12M_Flag'] = np.where((df['Paymt_hst_01'] > 90) | (df['Paymt_hst_02'] > 90) | (df['Paymt_hst_03'] > 90) | (df['Paymt_hst_04'] > 90) | (df['Paymt_hst_05'] > 90) | (df['Paymt_hst_06'] > 90)|(df['Paymt_hst_07'] > 90) | (df['Paymt_hst_08'] > 90) | (df['Paymt_hst_09'] > 90) | (df['Paymt_hst_10'] > 90) | (df['Paymt_hst_11'] > 90) | (df['Paymt_hst_12'] > 90) ,1,0)
    
    ## Mths since last delq
    df['Months_since_Last_Deliquent'] = np.where(df['Paymt_hst_01']>0,1,(np.where(df['Paymt_hst_02']>0,2,(np.where(df['Paymt_hst_03']>0,3,
                                        (np.where(df['Paymt_hst_04']>0,4,(np.where(df['Paymt_hst_05']>0,5,(np.where(df['Paymt_hst_06']>0,6,
                                        (np.where(df['Paymt_hst_07']>0,7,(np.where(df['Paymt_hst_08']>0,8,(np.where(df['Paymt_hst_09']>0,9,
                                        (np.where(df['Paymt_hst_10']>0,10,(np.where(df['Paymt_hst_11']>0,11,(np.where(df['Paymt_hst_12']>0,12,
                                        (np.where(df['Paymt_hst_13']>0,13,(np.where(df['Paymt_hst_14']>0,14,(np.where(df['Paymt_hst_15']>0,15,
                                        (np.where(df['Paymt_hst_16']>0,16,(np.where(df['Paymt_hst_17']>0,17,(np.where(df['Paymt_hst_18']>0,18,
                                        (np.where(df['Paymt_hst_19']>0,19,(np.where(df['Paymt_hst_20']>0,20,(np.where(df['Paymt_hst_21']>0,21,
                                        (np.where(df['Paymt_hst_22']>0,22,(np.where(df['Paymt_hst_23']>0,23,(np.where(df['Paymt_hst_24']>0,24,
                                        (np.where(df['Paymt_hst_25']>0,25,(np.where(df['Paymt_hst_26']>0,26,(np.where(df['Paymt_hst_27']>0,27,
                                        (np.where(df['Paymt_hst_28']>0,28,(np.where(df['Paymt_hst_29']>0,29,(np.where(df['Paymt_hst_30']>0,30,
                                        (np.where(df['Paymt_hst_31']>0,31,(np.where(df['Paymt_hst_32']>0,32,(np.where(df['Paymt_hst_33']>0,33,
                                        (np.where(df['Paymt_hst_34']>0,34,
                                        (np.where(df['Paymt_hst_35']>0,35,
                                        (np.where(df['Paymt_hst_36']>0,36,0)))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))
    
    ## Mths since last 30p
    df['Months_since_Last_Deliquent_G30'] = np.where(df['Paymt_hst_01']>=30,1,(np.where(df['Paymt_hst_02']>=30,2,(np.where(df['Paymt_hst_03']>=30,3,
                                        (np.where(df['Paymt_hst_04']>=30,4,(np.where(df['Paymt_hst_05']>=30,5,(np.where(df['Paymt_hst_06']>=30,6,
                                        (np.where(df['Paymt_hst_07']>=30,7,(np.where(df['Paymt_hst_08']>=30,8,(np.where(df['Paymt_hst_09']>=30,9,
                                        (np.where(df['Paymt_hst_10']>=30,10,(np.where(df['Paymt_hst_11']>=30,11,(np.where(df['Paymt_hst_12']>=30,12,
                                        (np.where(df['Paymt_hst_13']>=30,13,(np.where(df['Paymt_hst_14']>=30,14,(np.where(df['Paymt_hst_15']>=30,15,
                                        (np.where(df['Paymt_hst_16']>=30,16,(np.where(df['Paymt_hst_17']>=30,17,(np.where(df['Paymt_hst_18']>=30,18,
                                        (np.where(df['Paymt_hst_19']>=30,19,(np.where(df['Paymt_hst_20']>=30,20,(np.where(df['Paymt_hst_21']>=30,21,
                                        (np.where(df['Paymt_hst_22']>=30,22,(np.where(df['Paymt_hst_23']>=30,23,(np.where(df['Paymt_hst_24']>=30,24,
                                        (np.where(df['Paymt_hst_25']>=30,25,(np.where(df['Paymt_hst_26']>=30,26,(np.where(df['Paymt_hst_27']>=30,27,
                                        (np.where(df['Paymt_hst_28']>=30,28,(np.where(df['Paymt_hst_29']>=30,29,(np.where(df['Paymt_hst_30']>=30,30,
                                        (np.where(df['Paymt_hst_31']>=30,31,(np.where(df['Paymt_hst_32']>=30,32,(np.where(df['Paymt_hst_33']>=30,33,
                                        (np.where(df['Paymt_hst_34']>=30,34,
                                        (np.where(df['Paymt_hst_35']>=30,35,(np.where(df['Paymt_hst_36']>0,36,0)))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))
    
    ## Mths since last 60p
    df['Months_since_Last_Deliquent_G60'] = np.where(df['Paymt_hst_01']>=60,1,(np.where(df['Paymt_hst_02']>=60,2,(np.where(df['Paymt_hst_03']>=60,3,
                                        (np.where(df['Paymt_hst_04']>=60,4,(np.where(df['Paymt_hst_05']>=60,5,(np.where(df['Paymt_hst_06']>=60,6,
                                        (np.where(df['Paymt_hst_07']>=60,7,(np.where(df['Paymt_hst_08']>=60,8,(np.where(df['Paymt_hst_09']>=60,9,
                                        (np.where(df['Paymt_hst_10']>=60,10,(np.where(df['Paymt_hst_11']>=60,11,(np.where(df['Paymt_hst_12']>=60,12,
                                        (np.where(df['Paymt_hst_13']>=60,13,(np.where(df['Paymt_hst_14']>=60,14,(np.where(df['Paymt_hst_15']>=60,15,
                                        (np.where(df['Paymt_hst_16']>=60,16,(np.where(df['Paymt_hst_17']>=60,17,(np.where(df['Paymt_hst_18']>=60,18,
                                        (np.where(df['Paymt_hst_19']>=60,19,(np.where(df['Paymt_hst_20']>=60,20,(np.where(df['Paymt_hst_21']>=60,21,
                                        (np.where(df['Paymt_hst_22']>=60,22,(np.where(df['Paymt_hst_23']>=60,23,(np.where(df['Paymt_hst_24']>=60,24,
                                        (np.where(df['Paymt_hst_25']>=60,25,(np.where(df['Paymt_hst_26']>=60,26,(np.where(df['Paymt_hst_27']>=60,27,
                                        (np.where(df['Paymt_hst_28']>=60,28,(np.where(df['Paymt_hst_29']>=60,29,(np.where(df['Paymt_hst_30']>=60,30,
                                        (np.where(df['Paymt_hst_31']>=60,31,(np.where(df['Paymt_hst_32']>=60,32,(np.where(df['Paymt_hst_33']>=60,33,
                                        (np.where(df['Paymt_hst_34']>=60,34,
                                        (np.where(df['Paymt_hst_35']>=60,35,(np.where(df['Paymt_hst_36']>0,36,0)))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))
    
    ## mths since last 90p
    df['Months_since_Last_Deliquent_G90'] = np.where(df['Paymt_hst_01']>=90,1,(np.where(df['Paymt_hst_02']>=90,2,(np.where(df['Paymt_hst_03']>=90,3,
                                        (np.where(df['Paymt_hst_04']>=90,4,(np.where(df['Paymt_hst_05']>=90,5,(np.where(df['Paymt_hst_06']>=90,6,
                                        (np.where(df['Paymt_hst_07']>=90,7,(np.where(df['Paymt_hst_08']>=90,8,(np.where(df['Paymt_hst_09']>=90,9,
                                        (np.where(df['Paymt_hst_10']>=90,10,(np.where(df['Paymt_hst_11']>=90,11,(np.where(df['Paymt_hst_12']>=90,12,
                                        (np.where(df['Paymt_hst_13']>=90,13,(np.where(df['Paymt_hst_14']>=90,14,(np.where(df['Paymt_hst_15']>=90,15,
                                        (np.where(df['Paymt_hst_16']>=90,16,(np.where(df['Paymt_hst_17']>=90,17,(np.where(df['Paymt_hst_18']>=90,18,
                                        (np.where(df['Paymt_hst_19']>=90,19,(np.where(df['Paymt_hst_20']>=90,20,(np.where(df['Paymt_hst_21']>=90,21,
                                        (np.where(df['Paymt_hst_22']>=90,22,(np.where(df['Paymt_hst_23']>=90,23,(np.where(df['Paymt_hst_24']>=90,24,
                                        (np.where(df['Paymt_hst_25']>=90,25,(np.where(df['Paymt_hst_26']>=90,26,(np.where(df['Paymt_hst_27']>=90,27,
                                        (np.where(df['Paymt_hst_28']>=90,28,(np.where(df['Paymt_hst_29']>=90,29,(np.where(df['Paymt_hst_30']>=90,30,
                                        (np.where(df['Paymt_hst_31']>=90,31,(np.where(df['Paymt_hst_32']>=90,32,(np.where(df['Paymt_hst_33']>=90,33,
                                        (np.where(df['Paymt_hst_34']>=90,34,
                                        (np.where(df['Paymt_hst_35']>=90,35,(np.where(df['Paymt_hst_36']>0,36,0)))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))
    
    ## Mths since last mild delq
    df['Months_since_Last_Deliquent_L90'] = np.where((df['Paymt_hst_01']>=1)&(df['Paymt_hst_01']<=90),1,
                                        (np.where((df['Paymt_hst_02']>=1)&(df['Paymt_hst_02']<=90),2,
                                        (np.where((df['Paymt_hst_03']>=1)&(df['Paymt_hst_03']<=90),3,
                                        (np.where((df['Paymt_hst_04']>=1)&(df['Paymt_hst_04']<=90),4,
                                        (np.where((df['Paymt_hst_05']>=1)&(df['Paymt_hst_05']<=90),5,
                                        (np.where((df['Paymt_hst_06']>=1)&(df['Paymt_hst_06']<=90),6,
                                        (np.where((df['Paymt_hst_07']>=1)&(df['Paymt_hst_07']<=90),7,
                                        (np.where((df['Paymt_hst_08']>=1)&(df['Paymt_hst_08']<=90),8,
                                        (np.where((df['Paymt_hst_09']>=1)&(df['Paymt_hst_09']<=90),9,
                                        (np.where((df['Paymt_hst_10']>=1)&(df['Paymt_hst_10']<=90),10,
                                        (np.where((df['Paymt_hst_11']>=1)&(df['Paymt_hst_11']<=90),11,
                                        (np.where((df['Paymt_hst_12']>=1)&(df['Paymt_hst_12']<=90),12,
                                        (np.where((df['Paymt_hst_13']>=1)&(df['Paymt_hst_13']<=90),13,
                                        (np.where((df['Paymt_hst_14']>=1)&(df['Paymt_hst_14']<=90),14,
                                        (np.where((df['Paymt_hst_15']>=1)&(df['Paymt_hst_15']<=90),15,
                                        (np.where((df['Paymt_hst_16']>=1)&(df['Paymt_hst_16']<=90),16,
                                        (np.where((df['Paymt_hst_17']>=1)&(df['Paymt_hst_17']<=90),17,
                                        (np.where((df['Paymt_hst_18']>=1)&(df['Paymt_hst_18']<=90),18,
                                        (np.where((df['Paymt_hst_19']>=1)&(df['Paymt_hst_19']<=90),19,
                                        (np.where((df['Paymt_hst_20']>=1)&(df['Paymt_hst_20']<=90),20,
                                        (np.where((df['Paymt_hst_21']>=1)&(df['Paymt_hst_21']<=90),21,
                                        (np.where((df['Paymt_hst_22']>=1)&(df['Paymt_hst_22']<=90),22,
                                        (np.where((df['Paymt_hst_23']>=1)&(df['Paymt_hst_23']<=90),23,
                                        (np.where((df['Paymt_hst_24']>=1)&(df['Paymt_hst_24']<=90),24,
                                        (np.where((df['Paymt_hst_25']>=1)&(df['Paymt_hst_25']<=90),25,
                                        (np.where((df['Paymt_hst_26']>=1)&(df['Paymt_hst_26']<=90),26,
                                        (np.where((df['Paymt_hst_27']>=1)&(df['Paymt_hst_27']<=90),27,
                                        (np.where((df['Paymt_hst_28']>=1)&(df['Paymt_hst_28']<=90),28,
                                        (np.where((df['Paymt_hst_29']>=1)&(df['Paymt_hst_29']<=90),29,
                                        (np.where((df['Paymt_hst_30']>=1)&(df['Paymt_hst_30']<=90),30,
                                        (np.where((df['Paymt_hst_31']>=1)&(df['Paymt_hst_31']<=90),31,
                                        (np.where((df['Paymt_hst_32']>=1)&(df['Paymt_hst_32']<=90),32,
                                        (np.where((df['Paymt_hst_33']>=1)&(df['Paymt_hst_33']<=90),33,
                                        (np.where((df['Paymt_hst_34']>=1)&(df['Paymt_hst_34']<=90),34,
                                        (np.where((df['Paymt_hst_35']>=1)&(df['Paymt_hst_35']>=90),35,
                                        (np.where((df['Paymt_hst_36']>=1)&(df['Paymt_hst_36']>=90),36,0)))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))
    ## Max DPD in last 3/6/9/12 mths
    df['MAX_DPD_L3M'] = df[['Paymt_hst_01','Paymt_hst_02','Paymt_hst_03']].max(axis=1)
    df['MAX_DPD_L6M'] = df[['Paymt_hst_01','Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06']].max(axis=1)
    df['MAX_DPD_L9M'] = df[['Paymt_hst_01','Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09']].max(axis=1)
    df['MAX_DPD_L12M'] = df[['Paymt_hst_01', 'Paymt_hst_02','Paymt_hst_03','Paymt_hst_04','Paymt_hst_05','Paymt_hst_06','Paymt_hst_07','Paymt_hst_08','Paymt_hst_09','Paymt_hst_10','Paymt_hst_11','Paymt_hst_12']].max(axis=1)
    
    # num mild delq
    df['Mild_Delq_L3M'] = np.where((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01']<90)|(df['Paymt_hst_02']>=1) & (df['Paymt_hst_02']<90)|(df['Paymt_hst_03']>=1) & (df['Paymt_hst_03']<90),(df['Paymt_hst_01']<90,df['Paymt_hst_02']<90,df['Paymt_hst_03']<90),0).sum(axis=0)
    df['Mild_Delq_L6M'] = np.where((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01'] <90) | (df['Paymt_hst_02']>=1) & (df['Paymt_hst_02'] <90) | (df['Paymt_hst_03']>=1) & (df['Paymt_hst_03'] <90) | (df['Paymt_hst_04']>=1) & (df['Paymt_hst_04'] <90) | (df['Paymt_hst_05']>=1) & (df['Paymt_hst_05'] <90) | (df['Paymt_hst_06']>=1) & (df['Paymt_hst_06'] <90),(df['Paymt_hst_01'] <90,df['Paymt_hst_02'] <90,df['Paymt_hst_03'] <90,df['Paymt_hst_04'] <90,df['Paymt_hst_05'] <90,df['Paymt_hst_06'] <90),0).sum(axis=0)
    df['Mild_Delq_L9M'] = np.where((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01'] <90) | (df['Paymt_hst_02']>=1) & (df['Paymt_hst_02'] <90) | (df['Paymt_hst_03']>=1) & (df['Paymt_hst_03'] <90) | (df['Paymt_hst_04']>=1) & (df['Paymt_hst_04'] <90) | (df['Paymt_hst_05']>=1) & (df['Paymt_hst_05'] <90) | (df['Paymt_hst_06']>=1) &(df['Paymt_hst_06'] <90)|(df['Paymt_hst_07']>=1) & (df['Paymt_hst_07'] <90) | (df['Paymt_hst_08']>=1) & (df['Paymt_hst_08'] <90) | (df['Paymt_hst_09']>=1) & (df['Paymt_hst_09'] <90),(df['Paymt_hst_01'] <90,df['Paymt_hst_02'] <90,df['Paymt_hst_03'] <90,df['Paymt_hst_04'] <90,df['Paymt_hst_05'] <90,df['Paymt_hst_06'] <90,df['Paymt_hst_07'] <90,df['Paymt_hst_08'] <90,df['Paymt_hst_09'] <90),0).sum(axis=0)
    df['Mild_Delq_L12M'] = np.where((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01'] <90) | (df['Paymt_hst_02']>=1) & (df['Paymt_hst_02'] <90) | (df['Paymt_hst_03']>=1) & (df['Paymt_hst_03'] <90) | (df['Paymt_hst_04']>=1) & (df['Paymt_hst_04'] <90) | (df['Paymt_hst_05']>=1) & (df['Paymt_hst_05'] <90) | (df['Paymt_hst_06']>=1) & (df['Paymt_hst_06'] <90)|(df['Paymt_hst_07']>=1) & (df['Paymt_hst_07'] <90) | (df['Paymt_hst_08']>=1) & (df['Paymt_hst_08'] <90) | (df['Paymt_hst_09']>=1) & (df['Paymt_hst_09'] <90) | (df['Paymt_hst_10']>=1) & (df['Paymt_hst_10'] <90) | (df['Paymt_hst_11']>=1) & (df['Paymt_hst_11'] <90) | (df['Paymt_hst_12']>=1) & (df['Paymt_hst_12'] <90),(df['Paymt_hst_01'] <90,df['Paymt_hst_02'] <90,df['Paymt_hst_03'] <90,df['Paymt_hst_04'] <90,df['Paymt_hst_05'] <90,df['Paymt_hst_06'] <90,df['Paymt_hst_07'] <90,df['Paymt_hst_08'] <90,df['Paymt_hst_09'] <90,df['Paymt_hst_10'] <90,df['Paymt_hst_11'] <90,df['Paymt_hst_12'] <90),0).sum(axis=0)
    
    #num severe delq
    df['Severe_Delq_L3M'] = np.where((df['Paymt_hst_01']>=90)|(df['Paymt_hst_02']>=90)|(df['Paymt_hst_03']>=90),(df['Paymt_hst_01']>=90,df['Paymt_hst_02']>=90,df['Paymt_hst_03']>=90),0).sum(axis=0)
    df['Severe_Delq_L6M'] = np.where((df['Paymt_hst_01'] >=90) | (df['Paymt_hst_02'] >=90) | (df['Paymt_hst_03'] >=90) | (df['Paymt_hst_04'] >=90) | (df['Paymt_hst_05'] >=90) | (df['Paymt_hst_06'] >=90),(df['Paymt_hst_01'] >=90,df['Paymt_hst_02'] >=90,df['Paymt_hst_03'] >=90,df['Paymt_hst_04'] >=90,df['Paymt_hst_05'] >=90,df['Paymt_hst_06'] >=90),0).sum(axis=0)
    df['Severe_Delq_L9M'] = np.where((df['Paymt_hst_01'] >=90) | (df['Paymt_hst_02'] >=90) | (df['Paymt_hst_03'] >=90) | (df['Paymt_hst_04'] >=90) | (df['Paymt_hst_05'] >=90) | (df['Paymt_hst_06'] >=90)|(df['Paymt_hst_07'] >=90) | (df['Paymt_hst_08'] >=90) | (df['Paymt_hst_09'] >=90),(df['Paymt_hst_01'] >=90,df['Paymt_hst_02'] >=90,df['Paymt_hst_03'] >=90,df['Paymt_hst_04'] >=90,df['Paymt_hst_05'] >=90,df['Paymt_hst_06'] >=90,df['Paymt_hst_07'] >=90,df['Paymt_hst_08'] >=90,df['Paymt_hst_09'] >=90),0).sum(axis=0)
    df['Severe_Delq_L12M'] = np.where((df['Paymt_hst_01'] >=90) | (df['Paymt_hst_02'] >=90) | (df['Paymt_hst_03'] >=90) | (df['Paymt_hst_04'] >=90) | (df['Paymt_hst_05'] >=90) | (df['Paymt_hst_06'] >=90)|(df['Paymt_hst_07'] >=90) | (df['Paymt_hst_08'] >=90) | (df['Paymt_hst_09'] >=90) | (df['Paymt_hst_10'] >=90) | (df['Paymt_hst_11'] >=90) | (df['Paymt_hst_12'] >=90),(df['Paymt_hst_01'] >=90,df['Paymt_hst_02'] >=90,df['Paymt_hst_03'] >=90,df['Paymt_hst_04'] >=90,df['Paymt_hst_05'] >=90,df['Paymt_hst_06'] >=90,df['Paymt_hst_07'] >=90,df['Paymt_hst_08'] >=90,df['Paymt_hst_09'] >=90,df['Paymt_hst_10'] >=90,df['Paymt_hst_11'] >=90,df['Paymt_hst_12'] >=90),0).sum(axis=0)
    
    # mild delq L 3/6/9/12 flag
    df['Mild_Delq_L3M_Flag'] = np.where((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01']<90)|(df['Paymt_hst_02']>=1) & (df['Paymt_hst_02']<90)|(df['Paymt_hst_03']>=1) & (df['Paymt_hst_03']<90),1,0)
    df['Mild_Delq_L6M_Flag'] = np.where((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01'] <90) | (df['Paymt_hst_02']>=1) & (df['Paymt_hst_02'] <90) | (df['Paymt_hst_03']>=1) & (df['Paymt_hst_03'] <90) | (df['Paymt_hst_04']>=1) & (df['Paymt_hst_04'] <90) | (df['Paymt_hst_05']>=1) & (df['Paymt_hst_05'] <90) | (df['Paymt_hst_06']>=1) & (df['Paymt_hst_06'] <90),1,0)
    df['Mild_Delq_L9M_Flag'] = np.where((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01'] <90) | (df['Paymt_hst_02']>=1) & (df['Paymt_hst_02'] <90) | (df['Paymt_hst_03']>=1) & (df['Paymt_hst_03'] <90) | (df['Paymt_hst_04']>=1) & (df['Paymt_hst_04'] <90) | (df['Paymt_hst_05']>=1) & (df['Paymt_hst_05'] <90) | (df['Paymt_hst_06']>=1) &(df['Paymt_hst_06'] <90)|(df['Paymt_hst_07']>=1) & (df['Paymt_hst_07'] <90) | (df['Paymt_hst_08']>=1) & (df['Paymt_hst_08'] <90) | (df['Paymt_hst_09']>=1) & (df['Paymt_hst_09'] <90),1,0)
    df['Mild_Delq_L12M_Flag'] = np.where((df['Paymt_hst_01']>=1) & (df['Paymt_hst_01'] <90) | (df['Paymt_hst_02']>=1) & (df['Paymt_hst_02'] <90) | (df['Paymt_hst_03']>=1) & (df['Paymt_hst_03'] <90) | (df['Paymt_hst_04']>=1) & (df['Paymt_hst_04'] <90) | (df['Paymt_hst_05']>=1) & (df['Paymt_hst_05'] <90) | (df['Paymt_hst_06']>=1) & (df['Paymt_hst_06'] <90)|(df['Paymt_hst_07']>=1) & (df['Paymt_hst_07'] <90) | (df['Paymt_hst_08']>=1) & (df['Paymt_hst_08'] <90) | (df['Paymt_hst_09']>=1) & (df['Paymt_hst_09'] <90) | (df['Paymt_hst_10']>=1) & (df['Paymt_hst_10'] <90) | (df['Paymt_hst_11']>=1) & (df['Paymt_hst_11'] <90) | (df['Paymt_hst_12']>=1) & (df['Paymt_hst_12'] <90),1,0)
    
    # severe delq L 3/6/9/12 flag
    df['Severe_Delq_L3M_Flag'] = np.where((df['Paymt_hst_01']>=90)|(df['Paymt_hst_02']>=90)|(df['Paymt_hst_03']>=90),1,0)
    df['Severe_Delq_L6M_Flag'] = np.where((df['Paymt_hst_01'] >=90) | (df['Paymt_hst_02'] >=90) | (df['Paymt_hst_03'] >=90) | (df['Paymt_hst_04'] >=90) | (df['Paymt_hst_05'] >=90) | (df['Paymt_hst_06'] >=90),1,0)
    df['Severe_Delq_L9M_Flag'] = np.where((df['Paymt_hst_01'] >=90) | (df['Paymt_hst_02'] >=90) | (df['Paymt_hst_03'] >=90) | (df['Paymt_hst_04'] >=90) | (df['Paymt_hst_05'] >=90) | (df['Paymt_hst_06'] >=90)|(df['Paymt_hst_07'] >=90) | (df['Paymt_hst_08'] >=90) | (df['Paymt_hst_09'] >=90),1,0)
    df['Severe_Delq_L12M_Flag'] = np.where((df['Paymt_hst_01'] >=90) | (df['Paymt_hst_02'] >=90) | (df['Paymt_hst_03'] >=90) | (df['Paymt_hst_04'] >=90) | (df['Paymt_hst_05'] >=90) | (df['Paymt_hst_06'] >=90)|(df['Paymt_hst_07'] >=90) | (df['Paymt_hst_08'] >=90) | (df['Paymt_hst_09'] >=90) | (df['Paymt_hst_10'] >=90) | (df['Paymt_hst_11'] >=90) | (df['Paymt_hst_12'] >=90),1,0)
    
    # formating date columns
    df['DateProcessed'] = df['DateProcessed'].fillna(-1)
    df['DateProcessed'] = df['DateProcessed'].astype(int)
    df['DateProcessed'] = df['DateProcessed'].astype(str)
    df['DateProcessed'] = df['DateProcessed'].replace('-1', np.nan)
    df['DateProcessed'] =df['DateProcessed'].apply(lambda x: str(x) if not pd.isnull(x) else x)
    df['DateProcessed'] =df['DateProcessed'].apply(lambda x: x if not pd.isnull(x) else x)
    df['DateProcessed'] =df['DateProcessed'].apply(lambda x: x.zfill(8) if not pd.isnull(x) else x)
    df['DateProcessed_new'] =df['DateProcessed'].apply(lambda x : datetime.datetime.strptime(x, '%d%m%Y').strftime('%Y-%m-%d') if not pd.isnull(x) else x)
    df['DateProcessed_new'] =df['DateProcessed_new'].apply(lambda x: pd.to_datetime(x) if not pd.isnull(x) else x)
    
    df['DateClosed'] = df['DateClosed'].fillna(-1)
    df['DateClosed'] = df['DateClosed'].astype(int)
    df['DateClosed'] = df['DateClosed'].astype(str)
    df['DateClosed'] = df['DateClosed'].replace('-1', np.nan)
    df['DateClosed'] =df['DateClosed'].apply(lambda x: str(x) if not pd.isnull(x) else x)
    df['DateClosed'] =df['DateClosed'].apply(lambda x: x if not pd.isnull(x) else x)
    df['DateClosed'] =df['DateClosed'].apply(lambda x: x.zfill(8) if not pd.isnull(x) else x)
    df['DateClosed_new'] =df['DateClosed'].apply(lambda x : datetime.datetime.strptime(x, '%d%m%Y').strftime('%Y-%m-%d') if not pd.isnull(x) else x)
    df['DateClosed_new'] =df['DateClosed_new'].apply(lambda x: pd.to_datetime(x) if not pd.isnull(x) else x)
    
    df['DateReported_trades'] = df['DateReported_trades'].fillna(-1)
    df['DateReported_trades'] = df['DateReported_trades'].astype(int)
    df['DateReported_trades'] = df['DateReported_trades'].astype(str)
    df['DateReported_trades'] = df['DateReported_trades'].replace('-1', np.nan)
    df['DateReported_trades'] =df['DateReported_trades'].apply(lambda x: str(x) if not pd.isnull(x) else x)
    df['DateReported_trades'] =df['DateReported_trades'].apply(lambda x: x if not pd.isnull(x) else x)
    df['DateReported_trades'] =df['DateReported_trades'].apply(lambda x: x.zfill(8) if not pd.isnull(x) else x)
    df['DateReported_trades_new'] =df['DateReported_trades'].apply(lambda x : datetime.datetime.strptime(x, '%d%m%Y').strftime('%Y-%m-%d') if not pd.isnull(x) else x)
    df['DateReported_trades_new'] =df['DateReported_trades_new'].apply(lambda x: pd.to_datetime(x) if not pd.isnull(x) else x)
    
    df['DateOpenedDisbursed'] = df['DateOpenedDisbursed'].fillna(-1)
    df['DateOpenedDisbursed'] = df['DateOpenedDisbursed'].astype(int)
    df['DateOpenedDisbursed'] = df['DateOpenedDisbursed'].astype(str)
    df['DateOpenedDisbursed'] = df['DateOpenedDisbursed'].replace('-1', np.nan)
    df['DateOpenedDisbursed'] =df['DateOpenedDisbursed'].apply(lambda x: str(x) if not pd.isnull(x) else x)
    df['DateOpenedDisbursed'] =df['DateOpenedDisbursed'].apply(lambda x: x if not pd.isnull(x) else x)
    df['DateOpenedDisbursed'] =df['DateOpenedDisbursed'].apply(lambda x: x.zfill(8) if not pd.isnull(x) else x)
    df['DateOpenedDisbursed_new'] =df['DateOpenedDisbursed'].apply(lambda x : datetime.datetime.strptime(x, '%d%m%Y').strftime('%Y-%m-%d') if not pd.isnull(x) else x)
    df['DateOpenedDisbursed_new'] =df['DateOpenedDisbursed_new'].apply(lambda x: pd.to_datetime(x) if not pd.isnull(x) else x)
    
    ## Rollups
    
    df_rollup = pd.DataFrame()
    df_rollup['MemberReference'] =df['MemberReference'].unique()
    
    df['DateClosed_new_MissInd'] =np.where((np.isnan(df['DateClosed_new'])& pd.notnull(df['DateOpenedDisbursed_new'])) ,0,1)
    x1=df[(df['DateClosed_new_MissInd']==0)]
    x = x1.groupby(['MemberReference'])['DateClosed_new_MissInd'].count().reset_index()
    x = x.rename(columns={'DateClosed_new_MissInd': 'Num_active_open_TL'})
    del x1
    df_rollup = pd.merge(df_rollup, x, on='MemberReference',  how='left')
    del x
    
    
    #DateopenedorDisbursed - not blank 
    df['DatOpened_Disbursed_new_MissInd'] =np.where(pd.notnull(df['DateOpenedDisbursed_new']),1,0)
    
    
    #Number of Tradelines
    x1=df[(df['DatOpened_Disbursed_new_MissInd']==1)]
    x = x1.groupby(['MemberReference'])['DatOpened_Disbursed_new_MissInd'].count().reset_index()
    x = x.rename(columns={'DatOpened_Disbursed_new_MissInd': 'Num_of_Total_TL'})
    del x1
    
    df_rollup = pd.merge(df_rollup, x, on='MemberReference',  how='left')
    del x
    
    #CIBIL Variables
    df['score'] = df['score'].replace('000-1',1)
    df['score'] = df['score'].astype(float)
    df['CIBIL_Score_MissInd'] =np.where((pd.isnull(df['score'])),0,1)
    
    #Average CIBIL Score
    x1=df[(df['CIBIL_Score_MissInd']==1)]
    x = x1.groupby(['MemberReference'])['score'].mean().reset_index()
    x = x.rename(columns={'score': 'CIBIL_Score_Avg'})
    del x1
    df_rollup = pd.merge(df_rollup, x, on='MemberReference',  how='left')
    del x
    
    #Min CIBIL Score
    x1=df[(df['CIBIL_Score_MissInd']==1)]
    x = x1.groupby(['MemberReference'])['score'].min().reset_index()
    x = x.rename(columns={'score': 'CIBIL_Score_Min'})
    del x1
    
    df_rollup = pd.merge(df_rollup, x, on='MemberReference',  how='left')
    del x
    
    #Max CIBIL Score
    x1=df[(df['CIBIL_Score_MissInd']==1)]
    x = x1.groupby(['MemberReference'])['score'].max().reset_index()
    x = x.rename(columns={'score': 'CIBIL_Score_Max'})
    del x1
    df_rollup = pd.merge(df_rollup, x, on='MemberReference',  how='left')
    del x
    
    # overall outstanding bal
    x1=df[(df['DateClosed_new_MissInd']==0)]
    x = x1.groupby(['MemberReference'])['CurrentBalance'].sum().reset_index()
    x = x.rename(columns={'CurrentBalance': 'Overall_outstanding_Bal'})
    del x1
    df_rollup = pd.merge(df_rollup, x, on='MemberReference',  how='left')
    del x
    
    # Amount overdue    
    df['AmountOverdue']=df['AmountOverdue'].fillna(0)
    x = df.groupby(['MemberReference'])['AmountOverdue'].sum().reset_index()
    x = x.rename(columns={'AmountOverdue': 'AmountOverdue_Total'})
    df_rollup = pd.merge(df_rollup, x, on='MemberReference',  how='left')
    del x
    
    # amount overdue flag
    df_rollup['Amount_overdue_flag']= np.where(df_rollup['AmountOverdue_Total']!=0,1,0)
    
    # Months on bureau
    x1 = df.sort_values(by = ['MemberReference','DateReported_trades_new'],ascending = [True,True])
    x1 = x1.reset_index()
    del x1['index']
    x = x1.groupby(['MemberReference','DateProcessed_new'])['DateReported_trades_new'].min().reset_index()
    x = x.rename(columns={'DateReported_trades_new': 'Min_DateReported_trades_new'})
    del x1
    
    x['Month_on_bureau']=np.round((x['DateProcessed_new'] -x['Min_DateReported_trades_new'])/np.timedelta64(1,'M'))
    y = x[['MemberReference','Month_on_bureau']]
    df_rollup = pd.merge(df_rollup, y, on='MemberReference',  how='left')
    del x
    
    x = df.groupby(['MemberReference'])['num_times_Xp_L3M','num_times_Xp_L6M','num_times_Xp_L9M','num_times_Xp_L12M','num_times_30p_L3M','num_times_30p_L6M','num_times_30p_L9M','num_times_30p_L12M','num_times_60p_L3M','num_times_60p_L6M','num_times_60p_L9M','num_times_60p_L12M','num_times_90p_L3M','num_times_90p_L6M','num_times_90p_L9M','num_times_90p_L12M'].max().reset_index()
    
    df_rollup = pd.merge(df_rollup, x, on='MemberReference',  how='left')
    del x
    
    x = df.groupby(['MemberReference'])['Mild_Delq_L3M','Mild_Delq_L6M','Mild_Delq_L9M','Mild_Delq_L12M','Severe_Delq_L3M','Severe_Delq_L6M','Severe_Delq_L9M','Severe_Delq_L12M'].max().reset_index()
    df_rollup = pd.merge(df_rollup, x, on='MemberReference',  how='left')
    del x
    
    x = df.groupby(['MemberReference'])['Mild_Delq_L3M_Flag','Mild_Delq_L6M_Flag','Mild_Delq_L9M_Flag','Mild_Delq_L12M_Flag','Severe_Delq_L3M_Flag','Severe_Delq_L6M_Flag','Severe_Delq_L9M_Flag','Severe_Delq_L12M_Flag'].max().reset_index()
    df_rollup = pd.merge(df_rollup, x, on='MemberReference',  how='left')
    del x
    
    x = df.groupby(['MemberReference'])['MAX_DPD_L3M','MAX_DPD_L6M','MAX_DPD_L9M','MAX_DPD_L12M'].max().reset_index()
    df_rollup = pd.merge(df_rollup, x, on='MemberReference',  how='left')
    del x
    
    x = df.groupby(['MemberReference'])['X_DPD_L3M_Flag','X_DPD_L6M_Flag','X_DPD_L9M_Flag','X_DPD_L12M_Flag','30p_DPD_L3M_Flag','30p_DPD_L6M_Flag','30p_DPD_L9M_Flag','30p_DPD_L12M_Flag','60p_DPD_L3M_Flag','60p_DPD_L6M_Flag','60p_DPD_L9M_Flag','60p_DPD_L12M_Flag','90p_DPD_L3M_Flag','90p_DPD_L6M_Flag','90p_DPD_L9M_Flag','90p_DPD_L12M_Flag'].max().reset_index()
    df_rollup = pd.merge(df_rollup, x, on='MemberReference',  how='left')
    del x
    
    x = df.groupby(['MemberReference'])['Months_since_Last_Deliquent','Months_since_Last_Deliquent_G30','Months_since_Last_Deliquent_G60','Months_since_Last_Deliquent_G90','Months_since_Last_Deliquent_L90'].min().reset_index()
    df_rollup = pd.merge(df_rollup, x, on='MemberReference',  how='left')
    del x
    
    df_rollup = df_rollup.add_prefix('BUR_')
    df_rollup = df_rollup.rename(columns={'BUR_MemberReference':'INT_SZ_LOAN_ACCOUNT_NO'})
    
    
    return df_rollup

In [146]:
#test["BUR_Num_active_open_TL"]

In [147]:
Int_rollup_jul19 = df_mom_w_perf_jul19.add_prefix('INT_')

In [148]:
Int_rollup_oct19 = df_mom_w_perf_oct19.add_prefix('INT_')

In [149]:
Int_rollup_dec19 = df_mom_w_perf_dec19.add_prefix('INT_')

In [150]:
Int_w_bur_jul19 = pd.merge(Int_rollup_jul19,bur_rollup(bur_q1), how = "left", on = "INT_SZ_LOAN_ACCOUNT_NO", indicator = True )

In [151]:
Int_w_bur_oct19 = pd.merge(Int_rollup_oct19,bur_rollup(bur_q2), how = "left", on = "INT_SZ_LOAN_ACCOUNT_NO", indicator = True )

In [152]:
Int_w_bur_dec19 = pd.merge(Int_rollup_dec19,bur_rollup(bur_q3), how = "left", on = "INT_SZ_LOAN_ACCOUNT_NO", indicator = True )

In [153]:
Int_w_bur_jul19['_merge'].value_counts()/Int_w_bur_jul19['_merge'].count()

both          0.809555
left_only     0.190445
right_only    0.000000
Name: _merge, dtype: float64

In [154]:
Int_w_bur_dec19['_merge'].value_counts()/Int_w_bur_dec19['_merge'].count()

both          0.83923
left_only     0.16077
right_only    0.00000
Name: _merge, dtype: float64

In [155]:
Int_w_bur_oct19['_merge'].value_counts()/Int_w_bur_oct19['_merge'].count()

both          0.763396
left_only     0.236604
right_only    0.000000
Name: _merge, dtype: float64

In [156]:
Int_w_bur_jul19.drop("_merge",axis =1, inplace = True)
Int_w_bur_oct19.drop("_merge",axis =1, inplace = True)
Int_w_bur_dec19.drop("_merge",axis =1, inplace = True)


### Dev & OOT sample creation

In [157]:
dev_Jul_oct19 = pd.concat([Int_w_bur_jul19,Int_w_bur_oct19],axis = 0)

In [158]:
OOT_dec19 = Int_w_bur_dec19.copy()

## Missing value treatment 

In [159]:
dev_Jul_oct19.fillna(-99999, inplace=True)
dev_Jul_oct19.replace(np.inf, -99999, inplace=True)

In [160]:
OOT_dec19.fillna(-99999, inplace=True)
OOT_dec19.replace(np.inf, -99999, inplace=True)

## Retaining only required columns

In [161]:
dev_Jul_oct19.rename(columns = {"INT_xdays_ever_3m":"xdays_ever_3m"},inplace = True)
OOT_dec19.rename(columns = {"INT_xdays_ever_3m":"xdays_ever_3m"}, inplace = True)

In [168]:
dev_Jul_oct19_2 = dev_Jul_oct19.loc[:,['INT_Appl_datekey',
'INT_MOB',
'INT_I_NO_OF_PAID_EMI',
'INT_I_NO_OF_OVERDUE_EMI',
'INT_I_NO_OF_UNPAID_EMI',
'INT_TENOR',
'INT_LOAN_AMOUNT',
'INT_Times_30p_12m',
'INT_Times_60p_12m',
'INT_Times_90p_12m',
'INT_Times_xdays_12m',
'INT_Times_30p_9m',
'INT_Times_60p_9m',
'INT_Times_90p_9m',
'INT_Times_xdays_9m',
'INT_Times_30p_6m',
'INT_Times_60p_6m',
'INT_Times_90p_6m',
'INT_Times_xdays_6m',
'INT_Times_30p_3m',
'INT_Times_60p_3m',
'INT_Times_90p_3m',
'INT_Times_xdays_3m',
'INT_Last_12mths_30p_flag',
'INT_Last_12mths_60p_flag',
'INT_Last_12mths_90p_flag',
'INT_Last_12mths_xdays_flag',
'INT_Last_9mths_30p_flag',
'INT_Last_9mths_60p_flag',
'INT_Last_9mths_90p_flag',
'INT_Last_9mths_xdays_flag',
'INT_Last_6mths_30p_flag',
'INT_Last_6mths_60p_flag',
'INT_Last_6mths_90p_flag',
'INT_Last_6mths_xdays_flag',
'INT_Last_3mths_30p_flag',
'INT_Last_3mths_60p_flag',
'INT_Last_3mths_90p_flag',
'INT_Last_3mths_xdays_flag',
'INT_Max_dpd_Last_12m',
'INT_Max_dpd_Last_9m',
'INT_Max_dpd_Last_6m',
'INT_Max_dpd_Last_3m',
'INT_Times_late_paymt_12mths',
'INT_Times_late_paymt_9mths',
'INT_Times_late_paymt_6mths',
'INT_Times_late_paymt_3mths',
'INT_remaining_tenor',
'INT_paid_down_bal_ratio',
'INT_tenor_completed',
'INT_paymnts_made',
'INT_mths_since_30p',
'INT_mths_since_60p',
'INT_mths_since_90p',
'INT_Num_bounce_L3M',
'INT_Num_bounce_L6M',
'INT_Num_bounce_L9M',
'INT_Num_bounce_L12M',
'INT_Total_bounces',
'INT_perc_bounces',
'INT_school_locality',
'INT_SCH_BLD_OWN_FLAG',
'INT_num_land',
'INT_num_build',
'INT_land_flag',
'INT_building_flag',
'INT_both_L_B_flag',
'INT_LTV',
'BUR_Num_active_open_TL',
'BUR_Num_of_Total_TL',
'BUR_CIBIL_Score_Avg',
'BUR_CIBIL_Score_Min',
'BUR_CIBIL_Score_Max',
'BUR_Overall_outstanding_Bal',
'BUR_AmountOverdue_Total',
'BUR_Amount_overdue_flag',
'BUR_Month_on_bureau',
'BUR_num_times_Xp_L3M',
'BUR_num_times_Xp_L6M',
'BUR_num_times_Xp_L9M',
'BUR_num_times_Xp_L12M',
'BUR_num_times_30p_L3M',
'BUR_num_times_30p_L6M',
'BUR_num_times_30p_L9M',
'BUR_num_times_30p_L12M',
'BUR_num_times_60p_L3M',
'BUR_num_times_60p_L6M',
'BUR_num_times_60p_L9M',
'BUR_num_times_60p_L12M',
'BUR_num_times_90p_L3M',
'BUR_num_times_90p_L6M',
'BUR_num_times_90p_L9M',
'BUR_num_times_90p_L12M',
'BUR_Mild_Delq_L3M',
'BUR_Mild_Delq_L6M',
'BUR_Mild_Delq_L9M',
'BUR_Mild_Delq_L12M',
'BUR_Severe_Delq_L3M',
'BUR_Severe_Delq_L6M',
'BUR_Severe_Delq_L9M',
'BUR_Severe_Delq_L12M',
'BUR_Mild_Delq_L3M_Flag',
'BUR_Mild_Delq_L6M_Flag',
'BUR_Mild_Delq_L9M_Flag',
'BUR_Mild_Delq_L12M_Flag',
'BUR_Severe_Delq_L3M_Flag',
'BUR_Severe_Delq_L6M_Flag',
'BUR_Severe_Delq_L9M_Flag',
'BUR_Severe_Delq_L12M_Flag',
'BUR_MAX_DPD_L3M',
'BUR_MAX_DPD_L6M',
'BUR_MAX_DPD_L9M',
'BUR_MAX_DPD_L12M',
'BUR_X_DPD_L3M_Flag',
'BUR_X_DPD_L6M_Flag',
'BUR_X_DPD_L9M_Flag',
'BUR_X_DPD_L12M_Flag',
'BUR_30p_DPD_L3M_Flag',
'BUR_30p_DPD_L6M_Flag',
'BUR_30p_DPD_L9M_Flag',
'BUR_30p_DPD_L12M_Flag',
'BUR_60p_DPD_L3M_Flag',
'BUR_60p_DPD_L6M_Flag',
'BUR_60p_DPD_L9M_Flag',
'BUR_60p_DPD_L12M_Flag',
'BUR_90p_DPD_L3M_Flag',
'BUR_90p_DPD_L6M_Flag',
'BUR_90p_DPD_L9M_Flag',
'BUR_90p_DPD_L12M_Flag',
'BUR_Months_since_Last_Deliquent',
'BUR_Months_since_Last_Deliquent_G30',
'BUR_Months_since_Last_Deliquent_G60',
'BUR_Months_since_Last_Deliquent_G90',
'BUR_Months_since_Last_Deliquent_L90',
'xdays_ever_3m']]

In [169]:
OOT_dec19_2 = OOT_dec19.loc[:,['INT_Appl_datekey',
'INT_MOB',
'INT_I_NO_OF_PAID_EMI',
'INT_I_NO_OF_OVERDUE_EMI',
'INT_I_NO_OF_UNPAID_EMI',
'INT_TENOR',
'INT_LOAN_AMOUNT',
'INT_Times_30p_12m',
'INT_Times_60p_12m',
'INT_Times_90p_12m',
'INT_Times_xdays_12m',
'INT_Times_30p_9m',
'INT_Times_60p_9m',
'INT_Times_90p_9m',
'INT_Times_xdays_9m',
'INT_Times_30p_6m',
'INT_Times_60p_6m',
'INT_Times_90p_6m',
'INT_Times_xdays_6m',
'INT_Times_30p_3m',
'INT_Times_60p_3m',
'INT_Times_90p_3m',
'INT_Times_xdays_3m',
'INT_Last_12mths_30p_flag',
'INT_Last_12mths_60p_flag',
'INT_Last_12mths_90p_flag',
'INT_Last_12mths_xdays_flag',
'INT_Last_9mths_30p_flag',
'INT_Last_9mths_60p_flag',
'INT_Last_9mths_90p_flag',
'INT_Last_9mths_xdays_flag',
'INT_Last_6mths_30p_flag',
'INT_Last_6mths_60p_flag',
'INT_Last_6mths_90p_flag',
'INT_Last_6mths_xdays_flag',
'INT_Last_3mths_30p_flag',
'INT_Last_3mths_60p_flag',
'INT_Last_3mths_90p_flag',
'INT_Last_3mths_xdays_flag',
'INT_Max_dpd_Last_12m',
'INT_Max_dpd_Last_9m',
'INT_Max_dpd_Last_6m',
'INT_Max_dpd_Last_3m',
'INT_Times_late_paymt_12mths',
'INT_Times_late_paymt_9mths',
'INT_Times_late_paymt_6mths',
'INT_Times_late_paymt_3mths',
'INT_remaining_tenor',
'INT_paid_down_bal_ratio',
'INT_tenor_completed',
'INT_paymnts_made',
'INT_mths_since_30p',
'INT_mths_since_60p',
'INT_mths_since_90p',
'INT_Num_bounce_L3M',
'INT_Num_bounce_L6M',
'INT_Num_bounce_L9M',
'INT_Num_bounce_L12M',
'INT_Total_bounces',
'INT_perc_bounces',
'INT_school_locality',
'INT_SCH_BLD_OWN_FLAG',
'INT_num_land',
'INT_num_build',
'INT_land_flag',
'INT_building_flag',
'INT_both_L_B_flag',
'INT_LTV',
'BUR_Num_active_open_TL',
'BUR_Num_of_Total_TL',
'BUR_CIBIL_Score_Avg',
'BUR_CIBIL_Score_Min',
'BUR_CIBIL_Score_Max',
'BUR_Overall_outstanding_Bal',
'BUR_AmountOverdue_Total',
'BUR_Amount_overdue_flag',
'BUR_Month_on_bureau',
'BUR_num_times_Xp_L3M',
'BUR_num_times_Xp_L6M',
'BUR_num_times_Xp_L9M',
'BUR_num_times_Xp_L12M',
'BUR_num_times_30p_L3M',
'BUR_num_times_30p_L6M',
'BUR_num_times_30p_L9M',
'BUR_num_times_30p_L12M',
'BUR_num_times_60p_L3M',
'BUR_num_times_60p_L6M',
'BUR_num_times_60p_L9M',
'BUR_num_times_60p_L12M',
'BUR_num_times_90p_L3M',
'BUR_num_times_90p_L6M',
'BUR_num_times_90p_L9M',
'BUR_num_times_90p_L12M',
'BUR_Mild_Delq_L3M',
'BUR_Mild_Delq_L6M',
'BUR_Mild_Delq_L9M',
'BUR_Mild_Delq_L12M',
'BUR_Severe_Delq_L3M',
'BUR_Severe_Delq_L6M',
'BUR_Severe_Delq_L9M',
'BUR_Severe_Delq_L12M',
'BUR_Mild_Delq_L3M_Flag',
'BUR_Mild_Delq_L6M_Flag',
'BUR_Mild_Delq_L9M_Flag',
'BUR_Mild_Delq_L12M_Flag',
'BUR_Severe_Delq_L3M_Flag',
'BUR_Severe_Delq_L6M_Flag',
'BUR_Severe_Delq_L9M_Flag',
'BUR_Severe_Delq_L12M_Flag',
'BUR_MAX_DPD_L3M',
'BUR_MAX_DPD_L6M',
'BUR_MAX_DPD_L9M',
'BUR_MAX_DPD_L12M',
'BUR_X_DPD_L3M_Flag',
'BUR_X_DPD_L6M_Flag',
'BUR_X_DPD_L9M_Flag',
'BUR_X_DPD_L12M_Flag',
'BUR_30p_DPD_L3M_Flag',
'BUR_30p_DPD_L6M_Flag',
'BUR_30p_DPD_L9M_Flag',
'BUR_30p_DPD_L12M_Flag',
'BUR_60p_DPD_L3M_Flag',
'BUR_60p_DPD_L6M_Flag',
'BUR_60p_DPD_L9M_Flag',
'BUR_60p_DPD_L12M_Flag',
'BUR_90p_DPD_L3M_Flag',
'BUR_90p_DPD_L6M_Flag',
'BUR_90p_DPD_L9M_Flag',
'BUR_90p_DPD_L12M_Flag',
'BUR_Months_since_Last_Deliquent',
'BUR_Months_since_Last_Deliquent_G30',
'BUR_Months_since_Last_Deliquent_G60',
'BUR_Months_since_Last_Deliquent_G90',
'BUR_Months_since_Last_Deliquent_L90',
'xdays_ever_3m']]

In [177]:
dev_Jul_oct19_2['DEV_OOT']='DEV'
OOT_dec19_2['DEV_OOT'] = 'OOT'

In [504]:
#dev_Jul_oct19_2.to_csv('/home/siddarth/LMS_data/Input_CSV/dev_Jul_oct19_all.csv', index = False)
#OOT_dec19_2.to_csv('/home/siddarth/LMS_data/Input_CSV/OOT_dec19_all.csv',index = False)

### Adding 30p & 60p perf


In [172]:
df_30_3 = pd.read_csv("D:/Varthana/All_ss_30p_60p_9mths.csv")

In [178]:
dev_Jul_oct19_3 = pd.merge(dev_Jul_oct19_2,df_30_3, left_on = 'INT_Appl_datekey',
                          right_on = 'Appl_datekey', how = 'left' ).drop(['Unnamed: 0',
                                                                         'SZ_LOAN_ACCOUNT_NO',
                                                                          'Snapshot_dt',
                                                                         'Appl_datekey'],axis = 1)

In [179]:
OOT_dec19_2_3 = pd.merge(OOT_dec19_2,df_30_3, left_on = 'INT_Appl_datekey',
                          right_on = 'Appl_datekey', how = 'left' ).drop(['Unnamed: 0',
                                                                         'SZ_LOAN_ACCOUNT_NO',
                                                                         'Snapshot_dt',
                                                                         'Appl_datekey'],axis = 1)

In [176]:
list(dev_Jul_oct19_3)

['INT_Appl_datekey',
 'INT_MOB',
 'INT_I_NO_OF_PAID_EMI',
 'INT_I_NO_OF_OVERDUE_EMI',
 'INT_I_NO_OF_UNPAID_EMI',
 'INT_TENOR',
 'INT_LOAN_AMOUNT',
 'INT_Times_30p_12m',
 'INT_Times_60p_12m',
 'INT_Times_90p_12m',
 'INT_Times_xdays_12m',
 'INT_Times_30p_9m',
 'INT_Times_60p_9m',
 'INT_Times_90p_9m',
 'INT_Times_xdays_9m',
 'INT_Times_30p_6m',
 'INT_Times_60p_6m',
 'INT_Times_90p_6m',
 'INT_Times_xdays_6m',
 'INT_Times_30p_3m',
 'INT_Times_60p_3m',
 'INT_Times_90p_3m',
 'INT_Times_xdays_3m',
 'INT_Last_12mths_30p_flag',
 'INT_Last_12mths_60p_flag',
 'INT_Last_12mths_90p_flag',
 'INT_Last_12mths_xdays_flag',
 'INT_Last_9mths_30p_flag',
 'INT_Last_9mths_60p_flag',
 'INT_Last_9mths_90p_flag',
 'INT_Last_9mths_xdays_flag',
 'INT_Last_6mths_30p_flag',
 'INT_Last_6mths_60p_flag',
 'INT_Last_6mths_90p_flag',
 'INT_Last_6mths_xdays_flag',
 'INT_Last_3mths_30p_flag',
 'INT_Last_3mths_60p_flag',
 'INT_Last_3mths_90p_flag',
 'INT_Last_3mths_xdays_flag',
 'INT_Max_dpd_Last_12m',
 'INT_Max_dpd_Last_9

In [180]:
dev_Jul_oct19_3.shape, OOT_dec19_2_3.shape

((6378, 138), (3533, 138))

In [182]:
Final_data = pd.concat([dev_Jul_oct19_3,OOT_dec19_2_3])
Final_data.shape

(9911, 138)

In [184]:
Final_data['TENOR_COMPLETED'] = Final_data['INT_MOB']/Final_data['INT_TENOR']
Final_data[['INT_MOB','INT_TENOR','TENOR_COMPLETED']].head()

,INT_MOB,INT_TENOR,TENOR_COMPLETED
0,20.0,60.0,0.333333
1,23.0,36.0,0.638889
2,21.0,60.0,0.350000
3,22.0,60.0,0.366667
4,22.0,36.0,0.611111


In [188]:
Final_data['PROPERTY_TYPE'] = np.where(Final_data['INT_both_L_B_flag']>0,3,
                            np.where(Final_data['INT_land_flag']>0,2,
                            np.where(Final_data['INT_building_flag']>0,1,0)))
Final_data[['INT_both_L_B_flag','INT_land_flag','INT_building_flag','PROPERTY_TYPE']].head()

,INT_both_L_B_flag,INT_land_flag,INT_building_flag,PROPERTY_TYPE
0,-99999.0,-99999.0,-99999.0,0
1,-99999.0,-99999.0,-99999.0,0
2,-99999.0,-99999.0,-99999.0,0
3,-99999.0,-99999.0,-99999.0,0
4,-99999.0,-99999.0,-99999.0,0


In [190]:
Final_data.shape

(9911, 140)

In [191]:
Final_data.to_csv('D:/Varthana/Final_Data_Dev_OOT_V1.csv', index = False)
#dev_Jul_oct19_3.to_csv('D:/Varthana/dev_Jul_oct19_all_v5.csv', index = False)
#OOT_dec19_2_3.to_csv('D:/Varthana/OOT_dec19_all_v5.csv',index = False)